**Link to the 'data' folder containing all the .csv files used in this notebook**
https://drive.google.com/drive/folders/1yoFNldlv3zyhrLEpY58Gw2SDVm-MH50_?usp=sharing

**Add it to your drive shortcut** in a folder called Big Data. The path should be: directory = '/content/drive/My Drive/Big Data/data/'

#Police Killings Dataset - Team "How I Met Your Big Data"

##Imports, Drive Mounting and Function Definitions

In [1037]:
!pip install plotly

In [1038]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import random 
import time

from collections import defaultdict
from scipy.stats.stats import pearsonr
import pandas_profiling

In [1039]:
from google.colab import drive
drive.mount('/content/drive')
directory = '/content/drive/My Drive/Big Data/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1040]:
# Function to calculate missing values by column
def missing_values_table(df):

        # Total missing values
        mis_val = df.isnull().sum()

        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
          columns = {0 : 'Missing Values', 1 : '% of Total Values'}
        )
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
                '% of Total Values', ascending=False
            ).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values."
        )
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [1041]:
def info_column(col, more_info = True):
        # Column
        df_col = police[col]

        # Missing values
        miss_val = df_col.isnull().sum()
        miss_val_perc = round(miss_val * 100 / len(df_col), 1)

        # Unique values
        diff_values = len(df_col.unique())

        # Print formatted information 
        print(f"{col}: \n \tdtypes: {df_col.dtypes}\n\tNumber of missing values:\t{miss_val} ({miss_val_perc}%)\n\tNumber of unique values:\t{diff_values}\n")
  
        # In-depth analysis unique values  
        if more_info:
                if diff_values < 20:
                        item_counts = df_col.value_counts().rename('Value count')
                        item_counts_norm = round(df_col.value_counts(normalize=True)*100, 1).rename('%')
                        table = pd.concat([item_counts, item_counts_norm], axis=1)
                        return table
                else:
                        print('\t', df_col.unique()[:5], '...', df_col.unique()[-5:], '\n')


In [1042]:
def test_numeric(x):
    try:
        int(x)
        return True
    except Exception:
        return False

In [1043]:
def age_from_months(test_str):
  for ele in test_str.split():
    if ele.isdigit() < 23:
      age = 0
    else:
      age = 1
    return age

In [1044]:
def age_from_s(test_str):
  intero = int(test_str[:len(test_str)-1]) + 5
  return intero

In [1045]:
def trimester(x):
  if x <= 3:
    trimester = 1
  elif x > 3 and x <= 6:
    trimester = 2
  elif x > 6 and x <= 9:
    trimester = 3
  else:
    trimester = 4
  return trimester

In [1046]:
import geopy
from geopy.geocoders import Nominatim
  
# initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises2")
    
def get_location(lat, long, loc):
  location = geolocator.reverse(str(lat)+","+str(long), language='en', timeout=1)
  if location is None : return None
  else: 
    address = location.raw['address']
    if loc == 'county':
      loc = address.get('county', '')
    elif loc == 'city':
      loc = address.get('city', '')
  return loc


##Dataset Presentation

In [1047]:
police = pd.read_csv(directory+'police_fatalities.csv') 
police.head()

,Unique ID,Subject's name,Subject's age,Subject's gender,Subject's race,Subject's race with imputations,Imputation probability,URL of image of deceased,Date of injury resulting in death (month/day/year),Location of injury (address),Location of death (city),Location of death (state),Location of death (zip code),Location of death (county),Full Address,Latitude,Longitude,Agency responsible for death,Cause of death,A brief description of the circumstances surrounding the death,"Dispositions/Exclusions INTERNAL USE, NOT FOR ANALYSIS",Intentional Use of Force (Developing),Link to news article or photo of official document,"Symptoms of mental illness? INTERNAL USE, NOT FOR ANALYSIS",Video,Date&Description,Unique ID formula,Unique identifier (redundant),Date (Year)
0,25746,Samuel H. Knapp,17,Male,European-American/White,European-American/White,not imputed,NaN,01/01/2000,27898-27804 US-101,Willits,CA,95490.0,Mendocino,27898-27804 US-101 Willits CA 95490 Mendocino,39.470883,-123.361751,Mendocino County Sheriff's Office,Vehicle,Samuel Knapp was allegedly driving a stolen ve...,Unreported,Vehicle/Pursuit,https://drive.google.com/file/d/10DisrV8K5ReP1...,No,NaN,1/1/2000: Samuel Knapp was allegedly driving a...,NaN,25746.0,2000.0
1,25747,Mark A. Horton,21,Male,African-American/Black,African-American/Black,not imputed,NaN,01/01/2000,Davison Freeway,Detroit,MI,48203.0,Wayne,Davison Freeway Detroit MI 48203 Wayne,42.404526,-83.092274,NaN,Vehicle,Two Detroit men killed when their car crashed ...,Unreported,Vehicle/Pursuit,https://drive.google.com/file/d/1-nK-RohgiM-tZ...,No,NaN,1/1/2000: Two Detroit men killed when their ca...,NaN,25747.0,2000.0
2,25748,Phillip A. Blurbridge,19,Male,African-American/Black,African-American/Black,not imputed,NaN,01/01/2000,Davison Freeway,Detroit,MI,48203.0,Wayne,Davison Freeway Detroit MI 48203 Wayne,42.404526,-83.092274,NaN,Vehicle,Two Detroit men killed when their car crashed ...,Unreported,Vehicle/Pursuit,https://drive.google.com/file/d/1-nK-RohgiM-tZ...,No,NaN,1/1/2000: Two Detroit men killed when their ca...,NaN,25748.0,2000.0
3,25749,Mark Ortiz,23,Male,Hispanic/Latino,Hispanic/Latino,not imputed,NaN,01/01/2000,600 W Cherry Ln,Carlsbad,NM,88220.0,Eddy,600 W Cherry Ln Carlsbad NM 88220 Eddy,32.450080,-104.237643,Eddy County Sheriff's Office,Vehicle,A motorcycle was allegedly being driven errati...,Unreported,Vehicle/Pursuit,https://drive.google.com/file/d/1qAEefRjX_aTtC...,No,NaN,1/1/2000: A motorcycle was allegedly being dri...,NaN,25749.0,2000.0
4,2,Lester Miller,53,Male,Race unspecified,African-American/Black,0.947676492,NaN,01/02/2000,4850 Flakes Mill Road,Ellenwood,GA,30294.0,DeKalb,4850 Flakes Mill Road Ellenwood GA 30294 DeKalb,33.645164,-84.229413,DeKalb County Sheriff's Office,Gunshot,"Darren Mayfield, a DeKalb County sheriff's dep...",Criminal,"Intentional Use of Force, Deadly",https://docs.google.com/document/d/1-YuShSarW_...,No,NaN,"1/2/2000: Darren Mayfield, a DeKalb County she...",NaN,2.0,2000.0


The dataset (https://www.kaggle.com/djonafegnem/police-fatalities-in-the-us-from-2000-to-2020)
comes from https://fatalencounters.org/, and contains 29 features, most of which are quite self-explainatory. 

*   ``Unique ID``, ``Unique ID (redundant)`` - unique identifier for entries 
*   ``Subject's name`` - name of the subject
*   ``Subject's age`` - age of the subject 
*   ``Subject's gender`` - gender of the subject 
*   ``Subject's race``, ``Subject's race with imputations`` - fields indicating the ethnicity of the subject. 'Race with imputations' contains a prediction (done by an external organization) of the subject's race, for entries where the ethnicity was not reported
*   ``Imputation probability`` - Confidence for the imputation regarding the race (for entries which required such an estimation)
*   ``URL of image of deceased`` - image of the subject 
*   ``death_date_year`` - year of the death
*   ``Date of injury resulting in death`` - date when the subject got the injure that brought him to death
*   ``Location of injury (address)`` place where the subject was injured 
*   ``Location of death (city)``, ``(state)``, ``(zip code)``, ``(county)``, ``Full Address``, ``Latitude``, ``Longitude`` - informations regarding the place where the subject died
*   ``Agency responsible for death`` - police department (agency) responsible for the death of the subject 
*   ``Cause of death``, ``Description of the circumstances surrounding the death`` - causes/description that brought the subject to death
*   ``Dispositions/Exclusions`` The dataset labels this feature as "Not for analysis". It seems to contain information regarding the legal outcome of the case. More information below
*   ``Intentional Use of Force`` - Flagged as "developing". Not clear if it means "by the police officer(s)" or "by the subject".
*   ``death_article`` - url to the article(s) describing the death of the subject 
*   ``mental_illness`` - Whether the subject was diagnosed with/had symptoms of mental illness. The dataset labels this feature as "not for analysis" as well.
*   ``video`` - 28332 missing values, will be discarded
*   ``date_description`` - concatenation of date and death_description
*   ``unique_id_formula`` - 28333 missing values, will be deleted 

We **Rename** the columns with shorter names. </br>
We **Drop** some variables we already know will be useless or impractical to use effectively:

*   IDs
*   Links to photos, videos and news articles
*   Date_Description (redundant with date and description)

In [1048]:
police.rename(columns={
    "Unique ID": "id_1", 
    "Subject's name": "name",
    "Subject's age": "age",
    "Subject's gender": "gender",
    "Subject's race": "race",
    "Subject's race with imputations": "race_with_imputations",
    "Imputation probability": "imputation_probability", 
    "URL of image of deceased": "url_image",
    "Date of injury resulting in death (month/day/year)": "death_date", 
    "Location of injury (address)": 'death_address',
    "Location of death (city)": "death_city", 
    "Location of death (state)": "death_state",
    "Location of death (zip code)": "death_zipcode", 
    "Location of death (county)": "death_county", 
    "Full Address": "death_full_address", 
    "Latitude": "latitude",
    "Longitude": "longitude", 
    "Agency responsible for death": "police_agency",
    "Cause of death": "death_cause", 
    "A brief description of the circumstances surrounding the death": "death_description",
    "Dispositions/Exclusions INTERNAL USE, NOT FOR ANALYSIS": "dispositions_exclusions", 
    "Intentional Use of Force (Developing)": "use_of_force",
    "Link to news article or photo of official document": "death_article", 
    "Symptoms of mental illness? INTERNAL USE, NOT FOR ANALYSIS": "mental_illness",
    "Video": "video", 
    "Date&Description": "date_description",
    "Unique ID formula": "id_2", 
    "Unique identifier (redundant)": "id_3",
    "Date (Year)": "death_date_year", 
    }, inplace = True)


police.drop([
             'url_image', 
             'death_article', 
             'date_description', 
             'video', 
             'id_1',
             'id_2', 
             'id_3',
             'death_date_year'
             ], axis=1, inplace=True)

#The last row of the dataset was all NaNs
police.drop([28334], inplace=True)

The next table shows the presence of **Missing Values** in our dataset.
Please note that, since a lot of variables have missing values disguised in form of strings (ex: "Race Unspecified"), the actual missing values will be higher than those presented here. We will adress them column by column.

In [1049]:
missing_values_table(police)

Your selected dataframe has 21 columns.
There are 12 columns that have missing values.


,Missing Values,% of Total Values
age,980,3.5
death_address,542,1.9
imputation_probability,437,1.5
race_with_imputations,429,1.5
death_zipcode,189,0.7
gender,97,0.3
police_agency,68,0.2
mental_illness,61,0.2
death_city,35,0.1
death_county,18,0.1


##Data Integration

To augment the information at our disposal, we decided to integrate the original dataset using external sources, regarding demographical and political aspects of the 51 U.S. States.

### Population and Ethnic Distribution
Sources:

- population:
  https://data.ers.usda.gov/reports.aspx?ID=17827

- ethnic distributions:
  https://worldpopulationreview.com/states/states-by-race

- land area (for population density): https://it.wikipedia.org/wiki/Stati_degli_Stati_Uniti_d%27America_per_superficie
  



For each State, we retrieve the **number of its inhabitants** in 2000, 2010 and 2020.

In [1050]:
population = pd.read_csv(directory+'population.csv', sep=';')
population.drop(['Unnamed: 5', 'Unnamed: 6'], axis=1, inplace=True)
population.head()

,state,state_po,pop_2000,pop_2010,pop_2020
0,Alabama,AL,4.447.207,4.779.736,5.024.279
1,Alaska,AK,626.933,710.231,733.391
2,Arizona,AZ,5.130.247,6.392.017,7.151.502
3,Arkansas,AR,2.673.293,2.915.918,3.011.524
4,California,CA,33.871.653,37.253.956,39.538.223


In [1051]:
#remove the "." characters from the columns containing the population and cast them to int 
population['pop_2000'] = population['pop_2000'].str.replace('.', '').astype(int)
population['pop_2010'] = population['pop_2010'].str.replace('.', '').astype(int)
population['pop_2020'] = population['pop_2020'].str.replace('.', '').astype(int)
population.head()

,state,state_po,pop_2000,pop_2010,pop_2020
0,Alabama,AL,4447207,4779736,5024279
1,Alaska,AK,626933,710231,733391
2,Arizona,AZ,5130247,6392017,7151502
3,Arkansas,AR,2673293,2915918,3011524
4,California,CA,33871653,37253956,39538223


For each State, we retrieve its **Ethnic Distribution**: the percentages of its inhabitants belonging to a certain ethnicity ('*White*', '*Black*', '*Latino*', '*Indian*', '*Asian*', '*Hawaiian*'). 

In [1052]:
pop_perc = pd.read_csv(directory+'race.csv', sep=',')
pop_perc.drop(['Unnamed: 0', 'index', 'TotalPopulation'], axis=1, inplace=True)
pop_perc.head()

,Code,State,European-American/White,African-American/Black,Indian,Asian,Hawaiian,Other,Hispanic/Latino
0,AL,Alabama,0.6552,0.2649,0.0047,0.0134,0.0003,0.0187,0.0428
1,AK,Alaska,0.6063,0.0310,0.1442,0.0609,0.0122,0.0749,0.0705
2,AZ,Arizona,0.5471,0.0421,0.0392,0.0321,0.0018,0.0243,0.3134
3,AR,Arkansas,0.7243,0.1523,0.0059,0.0150,0.0029,0.0249,0.0747
4,CA,California,0.3718,0.0552,0.0036,0.1428,0.0036,0.0328,0.3902


Since three of these values ('*Indian*', '*Asian*', '*Hawaiian*') do not correspond perfectly to the ones found in the original dataset (where we have '*Asian/Pacific Islander*', '*Native American/Alaskan*' and '*Middle Eastern*'), we decide to simplify the information aggregating them into 'Other', since these are the less frequent values in our dataset (<1.5%) anyway. We will do the same when we'll address the Race column.

In [1053]:
pop_perc['other_2'] = pop_perc['Indian'] + pop_perc['Asian'] + pop_perc['Hawaiian'] + pop_perc['Other']
pop_perc.drop(['Indian', 'Asian', 'Hawaiian', 'Other'], axis=1, inplace=True)
pop_perc = pop_perc.rename(columns={'other_2': 'Other', 'Code' : 'state_po'})
pop_perc.head()

,state_po,State,European-American/White,African-American/Black,Hispanic/Latino,Other
0,AL,Alabama,0.6552,0.2649,0.0428,0.0371
1,AK,Alaska,0.6063,0.0310,0.0705,0.2922
2,AZ,Arizona,0.5471,0.0421,0.3134,0.0974
3,AR,Arkansas,0.7243,0.1523,0.0747,0.0487
4,CA,California,0.3718,0.0552,0.3902,0.1828


For each state, we retrive its **Area** in km2, to be able to compute its population density.

In [1054]:
landarea = pd.read_csv(directory+'landAreaKM.csv', sep=';')
landarea.drop(['position'], axis=1, inplace=True)
landarea.rename(columns={'state':'State'}, inplace=True)
landarea['areaKM'] = landarea['areaKM'].astype(int)
print(len(landarea))
landarea.head()

51


,State,areaKM
0,Alabama,135765
1,Alaska,1717854
2,Arizona,295234
3,Arkansas,137732
4,California,423970


Finally, we merge the information about **population**, **ethnic distribution** and **area** in a single dataframe and compute the population density of each state for 2020.

In [1055]:
landarea['State'].replace('Columbia', 'District of Columbia', inplace=True)
df_pop = population.merge(pop_perc, how='inner', on='state_po')
df_pop = df_pop.merge(landarea, how='inner', on='State')
df_pop.drop(['state'], axis=1, inplace=True)
df_pop['density2020'] = df_pop['pop_2020']/df_pop['areaKM']
df_pop_G = df_pop.set_index('state_po')
df_pop.head()

,state_po,pop_2000,pop_2010,pop_2020,State,European-American/White,African-American/Black,Hispanic/Latino,Other,areaKM,density2020
0,AL,4447207,4779736,5024279,Alabama,0.6552,0.2649,0.0428,0.0371,135765,37.007174
1,AK,626933,710231,733391,Alaska,0.6063,0.0310,0.0705,0.2922,1717854,0.426923
2,AZ,5130247,6392017,7151502,Arizona,0.5471,0.0421,0.3134,0.0974,295234,24.223165
3,AR,2673293,2915918,3011524,Arkansas,0.7243,0.1523,0.0747,0.0487,137732,21.865100
4,CA,33871653,37253956,39538223,California,0.3718,0.0552,0.3902,0.1828,423970,93.257124


### Politics and Presidential Elections
Source:

- presidential elections:
  https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ

From a dataset containing information about the election results in each county for the years 2000 to 2020, we want to extract the **party that won the elections in each state**. The objective is to eventually check whether the predominant political current in each state is correlated to the number of fatal encounters registered in a given time frame.

In [1056]:
pres = pd.read_csv('/content/drive/MyDrive/Big Data/data/countypres_2000-2020.csv')
pres.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,AL GORE,DEMOCRAT,4942.0,17208.0,20191203,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993.0,17208.0,20191203,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,RALPH NADER,GREEN,160.0,17208.0,20191203,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,OTHER,OTHER,113.0,17208.0,20191203,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003.0,PRESIDENT,AL GORE,DEMOCRAT,13997.0,56480.0,20191203,TOTAL


In [1057]:
pres['candidate'].replace('DONALD J TRUMP', 'DONALD TRUMP', inplace=True)
pres['candidate'].replace('JOSEPH R BIDEN JR', 'JOSEPH BIDEN', inplace=True)

In [1058]:
pres['candidate'].unique()

array(['AL GORE', 'GEORGE W. BUSH', 'RALPH NADER', 'OTHER', 'JOHN KERRY',
       'BARACK OBAMA', 'JOHN MCCAIN', 'MITT ROMNEY', 'HILLARY CLINTON',
       'DONALD TRUMP', 'JOSEPH BIDEN', 'JO JORGENSEN'], dtype=object)

In [1059]:
county_name = pres.groupby(['year', 'state_po', 'county_name'], as_index=False)['candidatevotes'].max()
county_merged = county_name.merge(pres, how='inner', on=['year', 'state_po', 'county_name', 'candidatevotes'])
county_merged.drop(['version', 'totalvotes', 'mode', 'county_fips', 'office', 'state'], axis=1, inplace=True)
county_merged.head()

,year,state_po,county_name,candidatevotes,candidate,party
0,2000,AK,DISTRICT 1,4681.0,GEORGE W. BUSH,REPUBLICAN
1,2000,AK,DISTRICT 10,5673.0,GEORGE W. BUSH,REPUBLICAN
2,2000,AK,DISTRICT 11,3960.0,GEORGE W. BUSH,REPUBLICAN
3,2000,AK,DISTRICT 12,4626.0,GEORGE W. BUSH,REPUBLICAN
4,2000,AK,DISTRICT 13,3853.0,GEORGE W. BUSH,REPUBLICAN


In [1060]:
state_comparison = county_merged.groupby(['year', 'state_po', 'party', 'candidate'], as_index=False).size()
state_comparison.head()

,year,state_po,party,candidate,size
0,2000,AK,DEMOCRAT,AL GORE,1
1,2000,AK,REPUBLICAN,GEORGE W. BUSH,39
2,2000,AL,DEMOCRAT,AL GORE,18
3,2000,AL,REPUBLICAN,GEORGE W. BUSH,49
4,2000,AR,DEMOCRAT,AL GORE,32


In [1061]:
state_winner = state_comparison.groupby(['year', 'state_po'], as_index=False)['size'].max()
state_winner = state_winner.merge(state_comparison, on=['year', 'state_po', 'size'])
state_winner.drop(['size'], axis=1, inplace=True)
state_winner.head()

,year,state_po,party,candidate
0,2000,AK,REPUBLICAN,GEORGE W. BUSH
1,2000,AL,REPUBLICAN,GEORGE W. BUSH
2,2000,AR,REPUBLICAN,GEORGE W. BUSH
3,2000,AZ,REPUBLICAN,GEORGE W. BUSH
4,2000,CA,REPUBLICAN,GEORGE W. BUSH


We also add two columns to record **who won the elections** overall in each year, so the name of the president and his party.

In [1062]:
conditions = [((state_winner['year'] == 2000) | (state_winner['year'] == 2004)), 
              ((state_winner['year'] == 2008) | (state_winner['year'] == 2012)), 
              (state_winner['year'] == 2016), (state_winner['year'] == 2020)]
values = ['GEORGE W. BUSH', 'BARACK OBAMA', 'DONALD TRUMP', 'JOSEPH BIDEN']
values2 = ['REPUBLICAN', 'DEMOCRAT', 'REPUBLICAN', 'DEMOCRAT'] 
state_winner['president'] = np.select(conditions, values)
state_winner['pres_party'] = np.select(conditions, values2)
state_winner.rename(columns={'party':'statewinner_party', 'candidate':'statewinner'}, inplace=True)
#example
state_winner[state_winner['state_po'] == 'CA']

,year,state_po,statewinner_party,statewinner,president,pres_party
4,2000,CA,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
55,2004,CA,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
106,2008,CA,DEMOCRAT,BARACK OBAMA,BARACK OBAMA,DEMOCRAT
157,2012,CA,DEMOCRAT,BARACK OBAMA,BARACK OBAMA,DEMOCRAT
208,2016,CA,DEMOCRAT,HILLARY CLINTON,DONALD TRUMP,REPUBLICAN
259,2020,CA,DEMOCRAT,JOSEPH BIDEN,JOSEPH BIDEN,DEMOCRAT


In the next cell we perform an operation useful to merge this additional dataframe with our original one: since in this one we have a row every 4 years for each state, we copy each row 3 more times and adjust the year value accordingly. In this way we can simply join on the *year* and the *state code*.

In [1063]:
for i, row in state_winner.iterrows():
  if(state_winner.at[i, 'year'] < 2020):
    for j in range(1,4):
      df_temp = {'year': state_winner.at[i,'year']+j, 'state_po': state_winner.at[i,'state_po'], 'statewinner_party': state_winner.at[i,'statewinner_party'], 
                 'statewinner': state_winner.at[i,'statewinner'], 'president': state_winner.at[i,'president'], 'pres_party': state_winner.at[i,'pres_party']}
      state_winner = state_winner.append(df_temp, ignore_index=True)

In [1064]:
state_winner = state_winner.sort_values('year')
state_winner[state_winner['state_po'] == 'CA']

,year,state_po,statewinner_party,statewinner,president,pres_party
4,2000,CA,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
319,2001,CA,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
320,2002,CA,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
321,2003,CA,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
55,2004,CA,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
472,2005,CA,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
473,2006,CA,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
474,2007,CA,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
106,2008,CA,DEMOCRAT,BARACK OBAMA,BARACK OBAMA,DEMOCRAT
625,2009,CA,DEMOCRAT,BARACK OBAMA,BARACK OBAMA,DEMOCRAT


# PREPROCESSING - Column by Column

## Age

The **info_column** function gives us, in a compact way, the number of missing and unique values for a given attribute. It also prints them in a table with the count and % of each value (but only for columns with 20 or less unique values, for readability reason). This function is most useful with categorical variables, which make up the most of our dataset.

In [1065]:
info_column('age')

age: 
 	dtypes: object
	Number of missing values:	980 (3.5%)
	Number of unique values:	119

	 ['17' '21' '19' '23' '53'] ... ['40-50' '97' '93' '4 months' '70s'] 



We have non-numeric values in this column that we need to address:

In [1066]:
unique, counts = np.unique(police.loc[~police['age'].apply(test_numeric), 'age'].tolist(), return_counts=True)
dict(zip(unique, counts))

{'10 months': 2,
 '18 months': 1,
 '18-25': 2,
 '2 months': 2,
 '20s': 17,
 '20s-30s': 1,
 '3 days': 1,
 '3 months': 3,
 '30s': 7,
 '4 months': 1,
 '40-50': 1,
 '40s': 4,
 '46/53': 1,
 '50s': 5,
 '55.': 1,
 '6 months': 1,
 '60s': 7,
 '7 months': 1,
 '70s': 1,
 '8 months': 1,
 '9 months': 1,
 'nan': 980}

"Age" contains around 1000 NaN values. This variable also contains ages expressed in months (for small children) and in ranges (for subjects of uncertain age). We decided to:

*  Transform ages like "X months" in 0 if X<12, 1 if X<24, etc.
*  Transform ages like "20s" in 24.
*  Transform ages like "18-25" in 21.

In [1067]:
police['age'].replace(['18-25'], 21, inplace=True)
police['age'].replace(['20s-30s'], 30, inplace=True)
police['age'].replace(['46/53'], 50, inplace=True)
police['age'].replace(['40-50','55.'], 55, inplace=True)

police.loc[police['age'].str.contains('months', na=False), 'age'] = police['age'].apply(lambda x: age_from_months(x) if isinstance(x, str) and 'months' in x else x)
police.loc[police['age'].str.contains('days', na=False), 'age'] = 0
police.loc[police['age'].str.contains('s', na=False), 'age'] = police.age.apply(lambda x: age_from_s(x) if isinstance(x, str) and 's' in x else x)

Age distribution (**before** filling missing values)

In [1068]:
fig = px.histogram(
    police.dropna(), 
    x = 'age', 
    nbins = 20
) 

fig.update_layout(
    title_text='Distributions of age (before filling missing values)',
    xaxis_title_text='Age', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1, # gap between bars of the same location coordinates
    width=800, height=400,
    legend = dict(
      orientation="h",
      yanchor="bottom",
      y=1,
      xanchor="right",
      x=1
    )
)

fig.show()

Filling of Missing Values. Since the original distribution of this variable is normal-like, we don't want to fill them with a single value. In order to avoid a spike in the distribution, we fill NaNs randomly following the pre-existing distribution.

In [1069]:
#AGE
np.random.seed(42) #!
#calculate the probability associated with each age value
s = police['age'].value_counts(normalize=True)
print('Value  '+'Normalized Frequency (probability)')
print(s)
#replacing missing values using the probability distribution
missing = police['age'].isnull()
police.loc[missing,'age'] = np.random.choice(s.index, size=len(police[missing]),p=s.values)

print(police['age'].isnull().sum())

Value  Normalized Frequency (probability)
22    0.033377
25    0.032536
24    0.031440
27    0.030891
23    0.030855
        ...   
50    0.000037
95    0.000037
30    0.000037
93    0.000037
97    0.000037
Name: age, Length: 107, dtype: float64
0


Age distribution (**after** filling missing values)

In [1070]:
fig = px.histogram(
    police,
    x = 'age', 
    nbins = 20
) 

fig.update_layout(
    title_text='Distributions of age (after filling missing values)', 
    xaxis_title_text='Age', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1, # gap between bars of the same location coordinates
    width=800, height=400,
    legend = dict(
      orientation="h",
      yanchor="bottom",
      y=1,
      xanchor="right",
      x=1
    )
)

fig.show()

We can see that the distribution remains more or less the same.

## Gender

In [1071]:
info_column('gender')

gender: 
 	dtypes: object
	Number of missing values:	97 (0.3%)
	Number of unique values:	4



,Value count,%
Male,25530,90.4
Female,2689,9.5
Transgender,18,0.1


Missing Values: ~~Since the mode is **very** prominent, we fill with it.~~

**UPDATE** Fill missing values using the Gender-Guesser library. https://pypi.org/project/gender-guesser/


In [1072]:
#Gender
#police['gender'].fillna('Male', inplace=True)
#police['gender'].isnull().sum()

In [1073]:
!pip install gender_guesser

In [1074]:
import gender_guesser.detector as gender 

def get_gender(name): 
  detector = gender.Detector()
  guess = detector.get_gender(name)
  if guess == 'female' or guess == 'mostly_female': 
    guess = 'Female'
    return guess
  else: 
    guess = 'Male'
    return guess

In [1075]:
#check manually
test = police.loc[police['gender'].isnull(), 'gender']
police.iloc[test.index]['name'].unique()

array(['Name withheld by police',
       'Marlon Keishawn Washington aka Marlon Kashun Washington',
       'Montalito McKissick', 'John Doe 6', 'John Doe 5', 'John Doe 4',
       'John Doe 3', 'John Doe 2', 'John Doe 1'], dtype=object)

Most of the rows where the gender was missing, were also missing the name. So we still fill them with "Male".

In [1076]:
np.vectorize(get_gender)(police[police['gender'].isnull()]['name'])

array(['Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male'], dtype='<U4')

In [1077]:
#Gender
police.loc[police['gender'].isnull(), 'gender'] = np.vectorize(get_gender)(police[police['gender'].isnull()]['name'])
police['gender'].fillna('Male', inplace=True)
police['gender'].isnull().sum()

0

In [1078]:
#Drop "Name"
police.drop(['name'], axis=1, inplace=True)

## Race, Race with imputations, Imputation Probability

**Race**

In [1079]:
info_column('race')

race: 
 	dtypes: object
	Number of missing values:	0 (0.0%)
	Number of unique values:	8



,Value count,%
European-American/White,9140,32.3
Race unspecified,8756,30.9
African-American/Black,6029,21.3
Hispanic/Latino,3647,12.9
Asian/Pacific Islander,432,1.5
Native American/Alaskan,281,1.0
Middle Eastern,48,0.2
HIspanic/Latino,1,0.0


First example in which we have a missing value disguised as a normal value: we transform "Race Unspecified" in NaN.

In [1080]:
police['race'].replace(['HIspanic/Latino'], 'Hispanic/Latino', inplace=True)
police['race'].replace(['Race unspecified'], np.nan, inplace=True)

**Race with imputations**

In [1081]:
info_column('race_with_imputations')

race_with_imputations: 
 	dtypes: object
	Number of missing values:	429 (1.5%)
	Number of unique values:	10



,Value count,%
European-American/White,13406,48.0
African-American/Black,7677,27.5
Hispanic/Latino,4587,16.4
Race unspecified,1372,4.9
Asian/Pacific Islander,529,1.9
Native American/Alaskan,276,1.0
Middle Eastern,48,0.2
Other Race,9,0.0
HIspanic/Latino,1,0.0


In [1082]:
police['race_with_imputations'].replace(['HIspanic/Latino'], 'Hispanic/Latino', inplace=True)
police['race_with_imputations'].replace(['Other Race'], 'Other', inplace=True)
police['race_with_imputations'].replace(['Race unspecified'], np.nan, inplace=True) 

Imputation_Probability contains the probability for the race imputation to be correct. The smallest probability is 30% (higher than a random choice among the 4 values), so we consider the imputed races to be good estimations. As such, we fill the missing values in "Race" with the values found in "Race_With_Imputations".

In [1083]:
police[['race', 'race_with_imputations', 'imputation_probability']].sort_values(by='imputation_probability').head()

,race,race_with_imputations,imputation_probability
790,NaN,European-American/White,0.295219178
25959,NaN,European-American/White,0.306219874
15478,NaN,Asian/Pacific Islander,0.316729248
680,NaN,European-American/White,0.31731393
14602,NaN,European-American/White,0.318614596


In [1084]:
police.race.fillna(police.race_with_imputations, inplace=True)

The number of missing values increased, even after filling with "Race with Imputations", because of the "Race Unspecified" value which we have cast to NaN.

In [1085]:
info_column('race')

race: 
 	dtypes: object
	Number of missing values:	1794 (6.3%)
	Number of unique values:	8



,Value count,%
European-American/White,13400,50.5
African-American/Black,7679,28.9
Hispanic/Latino,4595,17.3
Asian/Pacific Islander,527,2.0
Native American/Alaskan,282,1.1
Middle Eastern,48,0.2
Other,9,0.0


We don't need these 2 auxiliary columns anymore.

In [1086]:
police.drop(['race_with_imputations', 'imputation_probability'], axis=1, inplace=True)

As we said before, we cast *Asian/Pacific Islander*,  *Middle Eastern* and *Native American/Alaskan* to *Other* to simplify the dataset and to match the information found in the external dataframe regarding ethnic distributions.

In [1087]:
print(police['race'].unique())

['European-American/White' 'African-American/Black' 'Hispanic/Latino' nan
 'Asian/Pacific Islander' 'Native American/Alaskan' 'Other'
 'Middle Eastern']


In [1088]:
police['race'].replace(['Asian/Pacific Islander', 
                        'Native American/Alaskan', 
                        'Middle Eastern'], 'Other', inplace=True)

In [1089]:
print(police['race'].unique())

['European-American/White' 'African-American/Black' 'Hispanic/Latino' nan
 'Other']


**Missing Values**: Since we feel this column might be very important for our future task(s), we try to fill its missing values in a more sophisticated way, following the ethnic distribution of the state where the person with missing race died. The idea is "given a person in state X, which is the probability for that person to be white/black/hispanic/other?".

In [1090]:
states = df_pop['state_po'].unique() #list of states
ethnicdist = dict() #empty dict
print(len(states), states)

51 ['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'DC' 'CT' 'DE' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY']


Create a dictionary containing the ethnic distribution for each state

In [1091]:
for state in states: #create a dictionary where each state is associated with a pd.Series containing its ethnic distribution (from df_pop)
  ethnicdist[state] = pd.Series({'European-American/White': df_pop[df_pop['state_po']==state]['European-American/White'].values[0],
                       'African-American/Black': df_pop[df_pop['state_po']==state]['African-American/Black'].values[0],
                       'Hispanic/Latino': df_pop[df_pop['state_po']==state]['Hispanic/Latino'].values[0], 
                       'Other': df_pop[df_pop['state_po']==state]['Other'].values[0]})
#ex
ethnicdist['AL']

European-American/White    0.6552
African-American/Black     0.2649
Hispanic/Latino            0.0428
Other                      0.0371
dtype: float64

Use it to fill missing values

In [1092]:
for state in states: 
  missingrace = police[police['death_state']==state]['race'].isnull() #boolean series: rows with the current state and missing value for race have True
  police.loc[(police['death_state']==state) & (missingrace), 'race'] = np.random.choice(ethnicdist[state].index, 
                                                                                  size=len(police[police['death_state']==state][missingrace]),
                                                                                  p=ethnicdist[state].values)

In [1093]:
police['race'].isnull().sum()

0

## Temporal Information

###Date

In [1094]:
info_column('death_date')

death_date: 
 	dtypes: object
	Number of missing values:	0 (0.0%)
	Number of unique values:	7181

	 ['01/01/2000' '01/02/2000' '01/03/2000' '01/05/2000' '01/06/2000'] ... ['06/24/2020' '06/25/2020' '06/26/2020' '06/27/2020' '06/28/2020'] 



Transform dates in datetime objects (instead of simple strings)

In [1095]:
police['death_date'] = police['death_date'].apply(pd.to_datetime)

Plot fatal encounters per year:

In [1096]:
fig = px.histogram(police, x = 'death_date', nbins = 21) 

fig.update_layout(
    title_text='Death count per year (from 2000 to 2020)', # title of plot
    xaxis_title_text='Years', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    #bargroupgap=0.1 # gap between bars of the same location coordinates
    width=800, height=400
)

fig.show()

Attention! From this histogram it may seem that there have been less killing in 2020, BUT actually the dataset stops around the end of June 2020. It is more informative to show temporal information normalizing by the number of months recorded for a given year.

In [1097]:
month_mean_death = round(police.groupby(pd.Grouper(key='death_date',freq='M')).size()
                         .groupby(pd.Grouper(freq="Y")).mean(), 1)

In [1098]:
fig = px.bar(month_mean_death, x = pd.DatetimeIndex(month_mean_death.index).year, y = month_mean_death.values) 

fig.update_layout(
    title_text='Average death count per month over the years', # title of plot
    xaxis_title_text='Years', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    #bargroupgap=0.1 # gap between bars of the same location coordinates
    width=800, height=400
)

fig.show()

We can see that, in fact, fatal encounters have increased in 2020.

Overall, the number of deaths has been increasing since 2000: however, we have to keep in mind that this trend might appear simply because in the 00's (and early 10's) less fatal encounters were recorded in datasets like this.

### Trimester
We create an additional "Trimester" column for ease of representing temporal info. 

In [1099]:
police['trimester'] = police.death_date.dt.month.apply(lambda x: trimester(x))

In [1100]:
police[['death_date','trimester']].sort_values(by='death_date', ascending=False).head()

,death_date,trimester
28333,2020-06-28,2
28328,2020-06-27,2
28324,2020-06-27,2
28326,2020-06-27,2
28327,2020-06-27,2


In [1101]:
fig = px.histogram(police.loc[police['death_date'].dt.year == 2001], x = 'trimester') 

fig.update_layout( 
    title_text = 'Distribution of deaths by trimester in 2001 ', # title of plot
    xaxis_title_text = 'Years', # xaxis label
    yaxis_title_text = 'Count', # yaxis label
    bargap = 0.2, # gap between bars of adjacent location coordinates
    #bargroupgap=0.1 # gap between bars of the same location coordinates
    width = 600, height = 400
)

fig.show()

### Time (Feature Engineering)
We want to see if it is possible to retrieve information about the HOUR in which the fatal encounter happened from the "death_description" field. To do so, we search for "a.m" and "p.m." through a Regular Expression.

In [1102]:
police['time'] = police["death_description"].str.extract(r'(\d+(\s?:\s?\d+)?\s?((?:a|p|A|P)\.?(m|M)\.?))|(\d+(\s?:\s?\d+))')[0]

In [1103]:
info_column('time')

time: 
 	dtypes: object
	Number of missing values:	23600 (83.3%)
	Number of unique values:	729

	 ['12:35 a.m.' '1:54 a.m.' nan '2:30 a.m.' '11 p.m.'] ... ['5:11 a.m.' '11:41 a.m.' '8:56 a.m.' '11:08 p.m.' '7:08 p.m.'] 



Unfortunately less than 17% of the rows contain the info we have been seeking, so we have to give up our hopes of creating this variable.

In [1104]:
police.drop(['time'], axis=1, inplace=True)

## Geographical Information
We have a lot of spatial information!

In [1105]:
police[['death_address', 'death_city', 'death_state', 'death_zipcode', 'death_county', 'death_full_address', 'latitude', 'longitude']].head()

,death_address,death_city,death_state,death_zipcode,death_county,death_full_address,latitude,longitude
0,27898-27804 US-101,Willits,CA,95490.0,Mendocino,27898-27804 US-101 Willits CA 95490 Mendocino,39.470883,-123.361751
1,Davison Freeway,Detroit,MI,48203.0,Wayne,Davison Freeway Detroit MI 48203 Wayne,42.404526,-83.092274
2,Davison Freeway,Detroit,MI,48203.0,Wayne,Davison Freeway Detroit MI 48203 Wayne,42.404526,-83.092274
3,600 W Cherry Ln,Carlsbad,NM,88220.0,Eddy,600 W Cherry Ln Carlsbad NM 88220 Eddy,32.450080,-104.237643
4,4850 Flakes Mill Road,Ellenwood,GA,30294.0,DeKalb,4850 Flakes Mill Road Ellenwood GA 30294 DeKalb,33.645164,-84.229413


How many missing values we have?

In [1106]:
missing_values_table(police[['death_address', 'death_city', 'death_state', 'death_zipcode', 'death_county', 'death_full_address', 'latitude', 'longitude']])

Your selected dataframe has 8 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
death_address,542,1.9
death_zipcode,189,0.7
death_city,35,0.1
death_county,18,0.1


We drop the following columns because they contain information of too small granularity with respect to the analysis we expect to perform.

In [1107]:
police.drop(['death_address', 'death_zipcode', 'death_full_address'], axis=1, inplace=True)

The remaining information is about *City*, *County* and *State*.

In [1108]:
info_column('death_city', False)
info_column('death_county', False)
info_column('death_state', False)

death_city: 
 	dtypes: object
	Number of missing values:	35 (0.1%)
	Number of unique values:	5941

death_county: 
 	dtypes: object
	Number of missing values:	18 (0.1%)
	Number of unique values:	1517

death_state: 
 	dtypes: object
	Number of missing values:	0 (0.0%)
	Number of unique values:	51



Fortunately, *Latitude* and *Longitude* do not contain missing values. In this way we can use the geopy library to perform a reverse geocoding operation, and infer the missing values for county and city using the geographical coordinates.

In [1109]:
police.loc[police['death_county'].isnull(), 'death_county'] = np.vectorize(get_location)(police[police['death_county'].isnull()]['latitude'], 
                                                                                     police[police['death_county'].isnull()]['longitude'], 'county')

police.loc[police['death_city'].isnull(), 'death_city'] = np.vectorize(get_location)(police[police['death_city'].isnull()]['latitude'], 
                                                                                     police[police['death_city'].isnull()]['longitude'], 'city')


In [1110]:
police['death_city'] = police['death_city'].str.capitalize()
police['death_county'] = police['death_county'].str.capitalize()
police['death_state'] = police['death_state'].str.upper()

In [1111]:
info_column('death_city', False)
info_column('death_county', False)
info_column('death_state', False)

death_city: 
 	dtypes: object
	Number of missing values:	0 (0.0%)
	Number of unique values:	5923

death_county: 
 	dtypes: object
	Number of missing values:	0 (0.0%)
	Number of unique values:	1518

death_state: 
 	dtypes: object
	Number of missing values:	0 (0.0%)
	Number of unique values:	51



With the **folium** library we saw during our lectures, we plot an interactive heatmap of the fatal encounters. You can play around and zoom in to see exactly where each event happened. 

In [1112]:
import folium
from folium import plugins 

f = folium.Figure(width=1500, height=700)

##visualize data: we use a heatmap to check where data is refers to
map = folium.Map(
    location=[police.latitude.mean(), police.longitude.mean()], # where the map must be centered
    tiles='Stamen Toner', # the type of map to load
    zoom_start=4.7
)

hm = plugins.HeatMap(
    zip(police.latitude, police.longitude),
    min_opacity=0.2,
    radius=8, blur=15,
    max_zoom=1
)

map.add_child(hm)
map.add_to(f)

f

## Police_agency

This column contains information regarding the police department/office involved in the case. 

In [1113]:
info_column('police_agency')

police_agency: 
 	dtypes: object
	Number of missing values:	68 (0.2%)
	Number of unique values:	6297

	 ["Mendocino County Sheriff's Office" nan "Eddy County Sheriff's Office"
 "DeKalb County Sheriff's Office" 'Sacramento Police Department'] ... ["Stearns County Sheriff's Office" 'Auburndale Police Department'
 'Sun Prairie Police Department' 'Boone County Sheriff’s Office'
 "Red Bluff Police Department, Tehama County Sheriff's Office"] 



In [1114]:
police.police_agency.value_counts().head()

Los Angeles Police Department              469
Chicago Police Department                  423
Los Angeles County Sheriff's Department    345
City of New York Police Department         326
Houston Police Department                  310
Name: police_agency, dtype: int64

This variable does not give us any meaningful additional information that we cannot get from the various geographical variables, so we decide to drop it.

In [1115]:
police.drop(['police_agency'], axis=1, inplace=True)

## Death_cause

In [1116]:
info_column('death_cause')

death_cause: 
 	dtypes: object
	Number of missing values:	0 (0.0%)
	Number of unique values:	16



,Value count,%
Gunshot,20094,70.9
Vehicle,5803,20.5
Tasered,908,3.2
Medical emergency,381,1.3
Asphyxiated/Restrained,275,1.0
Drug overdose,175,0.6
Beaten/Bludgeoned with instrument,174,0.6
Drowned,172,0.6
Undetermined,102,0.4
Fell from a height,73,0.3


We replace the last two "rogue" values with their more frequent equivalent

In [1117]:
police['death_cause'].replace(['Unknown','Undetermined'], np.nan, inplace=True)
police['death_cause'].replace('Pursuit', 'Vehicle', inplace=True)

In [1118]:
info_column('death_cause', False)

death_cause: 
 	dtypes: object
	Number of missing values:	103 (0.4%)
	Number of unique values:	14



Missing Values: Since the mode is very prevalent, we fill using it.

In [1119]:
#death_cause
police['death_cause'].fillna('Gunshot', inplace=True)
police['death_cause'].isnull().sum()

0

## Death_description

In [1120]:
info_column('death_description', False)

death_description: 
 	dtypes: object
	Number of missing values:	0 (0.0%)
	Number of unique values:	26910



May not be useful computationally, but people who died in the same accident have the exact same description. It may help us group deaths caused by the same event.

In [1121]:
police.death_description

0        Samuel Knapp was allegedly driving a stolen ve...
1        Two Detroit men killed when their car crashed ...
2        Two Detroit men killed when their car crashed ...
3        A motorcycle was allegedly being driven errati...
4        Darren Mayfield, a DeKalb County sheriff's dep...
                               ...                        
28329    Bonnie Figueroa-Ortiz was reported missing. De...
28330    A Miami-Dade police officer in a marked cruise...
28331    Around 10:30 a.m., a Prince George officer clo...
28332    Around 10:30 a.m., a Prince George officer clo...
28333    Around 3 a.m., there was a crash at Fifth and ...
Name: death_description, Length: 28334, dtype: object

We keep this variable only to maintain some kind of descriptive information about the deaths, which may be useful to check on some results.

## Dispositions_exclusions

In [1122]:
info_column('dispositions_exclusions', False)
police.dispositions_exclusions.value_counts().head(50)

dispositions_exclusions: 
 	dtypes: object
	Number of missing values:	4 (0.0%)
	Number of unique values:	97



Unreported                                                               10353
Pending investigation                                                     6021
Justified                                                                 3992
Suicide                                                                   2516
Criminal                                                                  1252
Grand jury/No bill or Cleared                                              736
Unknown                                                                    700
Justified by District Attorney                                             406
Ruled suicide                                                              403
Cleared                                                                    398
Accidental                                                                 192
Murder/suicide                                                             172
Other justified (Civilian board/Prosecutor/District 

Since this variable has a very high cardinality, we try to aggregate these values according to their semantics.

In [1123]:
police['dispositions_exclusions'] = police['dispositions_exclusions'].str.title()
police.loc[police['dispositions_exclusions'].str.contains('Justified|Jusified|Justifed|Excusable|Accidental|Acquitted|Cleared|Dismissed|Medical Emergency|Settled Out Of Court', na=False), 'dispositions_exclusions'] = 'Justified'

police.loc[police['dispositions_exclusions'].str.contains('Criminal|Guilty|Administrative Discipline|Unjustified', na=False), 'dispositions_exclusions'] = 'Guilty'

police.loc[police['dispositions_exclusions'].str.contains('Suicide|Overdose', na=False), 'dispositions_exclusions'] = 'Suicide'

police.loc[police['dispositions_exclusions'].str.contains('Pending', na=False), 'dispositions_exclusions'] = 'Pending'

police['dispositions_exclusions'] = police['dispositions_exclusions'].str.split('\/').str[-1].str.strip()
police['dispositions_exclusions'].replace(['Unknown','Family Awarded Money','Results Unreported','Referred To Prosecutor','Drowned','Off-Duty'], np.nan, inplace=True)


In [1124]:
info_column('dispositions_exclusions', True)
#police.dispositions_exclusions.value_counts().head(23)

dispositions_exclusions: 
 	dtypes: object
	Number of missing values:	841 (3.0%)
	Number of unique values:	6



,Value count,%
Unreported,10353,37.7
Justified,6531,23.8
Pending,6072,22.1
Suicide,3140,11.4
Guilty,1397,5.1


## Use_of_force

In [1125]:
info_column('use_of_force')

use_of_force: 
 	dtypes: object
	Number of missing values:	1 (0.0%)
	Number of unique values:	12



,Value count,%
"Intentional Use of Force, Deadly",17105,60.4
Vehicle/Pursuit,5734,20.2
Suicide,2924,10.3
Yes,1640,5.8
No,662,2.3
"Intenional Use of Force, Deadly",175,0.6
Undetermined,48,0.2
Pursuit,40,0.1
Vehicle,3,0.0
Intentional Use of Force,1,0.0


In [1126]:
police['use_of_force'] = police['use_of_force'].str.capitalize()

In [1127]:
police['use_of_force'].replace(['Undetermined', 'Unknown'], np.nan, inplace=True)

In [1128]:
police.loc[police['use_of_force'].str.contains('use of force', na=False), 'use_of_force'] = 'Yes' 
police.loc[police['use_of_force'].str.contains('Vehicle|Pursuit', na=False), 'use_of_force'] = 'Vehicle/Pursuit' 

In [1129]:
info_column('use_of_force')

use_of_force: 
 	dtypes: object
	Number of missing values:	50 (0.2%)
	Number of unique values:	5



,Value count,%
Yes,18921,66.9
Vehicle/Pursuit,5777,20.4
Suicide,2924,10.3
No,662,2.3


Missing Values: fill with the mode.

In [1130]:
#Use_of_Force
police['use_of_force'] = police['use_of_force'].fillna('Yes')  

In [1131]:
police['use_of_force'].isnull().sum()

0

## Mental_illness

In [1132]:
info_column('mental_illness')

mental_illness: 
 	dtypes: object
	Number of missing values:	61 (0.2%)
	Number of unique values:	5



,Value count,%
No,18855,66.7
Unknown,4732,16.7
Yes,3222,11.4
Drug or alcohol use,1464,5.2


In [1133]:
police['mental_illness'].replace('Unknown', np.nan, inplace=True)

In [1134]:
info_column('mental_illness', False)

mental_illness: 
 	dtypes: object
	Number of missing values:	4793 (16.9%)
	Number of unique values:	4



Missing Values: fill with the mode.

In [1135]:
#mental_illness
police['mental_illness'].fillna('No', inplace=True)
police['mental_illness'].isnull().sum()

0

# Summarizing

In [1136]:
police_desc = police

police = police.reindex(columns=['age', 'gender', 'race', 'mental_illness', # who
                                 'death_cause', 'use_of_force', # how
                                 'death_date', 'trimester', # when
                                 'death_city', 'death_county', 'death_state', 'latitude', 'longitude', 'death_description', 'dispositions_exclusions']) # where
                                

In [1137]:
police.head()

,age,gender,race,mental_illness,death_cause,use_of_force,death_date,trimester,death_city,death_county,death_state,latitude,longitude,death_description,dispositions_exclusions
0,17,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,Willits,Mendocino,CA,39.470883,-123.361751,Samuel Knapp was allegedly driving a stolen ve...,Unreported
1,21,Male,African-American/Black,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,Detroit,Wayne,MI,42.404526,-83.092274,Two Detroit men killed when their car crashed ...,Unreported
2,19,Male,African-American/Black,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,Detroit,Wayne,MI,42.404526,-83.092274,Two Detroit men killed when their car crashed ...,Unreported
3,23,Male,Hispanic/Latino,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,Carlsbad,Eddy,NM,32.450080,-104.237643,A motorcycle was allegedly being driven errati...,Unreported
4,53,Male,African-American/Black,No,Gunshot,Yes,2000-01-02,1,Ellenwood,Dekalb,GA,33.645164,-84.229413,"Darren Mayfield, a DeKalb County sheriff's dep...",Guilty


In [1138]:
missing_values_table(police)

Your selected dataframe has 15 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
dispositions_exclusions,841,3.0


# Plots

## Who? Age, Gender and Race

In [1139]:
fig_age1 = px.histogram(
    police, # dropna per plot
    x = 'age', color = 'gender', barmode = 'group',
    nbins = 20
) 

fig_age1.update_layout(
    title_text='Distribution of age and gender', #by gender', # title of plot
    xaxis_title_text='Age', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1, # gap between bars of the same location coordinates
    width=800, height=400,
    legend = dict(
      orientation="h",
      yanchor="bottom",
      y=1,
      xanchor="right",
      x=1
    )
)

fig_age1.show()

In [1140]:
fig_age2 = px.histogram(
    police, # dropna per plot
    x = 'race', 
    color = 'gender', barmode = 'group', facet_col='gender', 
    histnorm = 'percent'
)\
.for_each_trace(lambda t: t.update(name=t.name.replace('gender=', " ")))

fig_age2.for_each_xaxis(lambda x: x.update(title = ''))

fig_age2.update_layout(
    title_text='Distribution of race by age and gender', #by gender', # title of plot
    yaxis_title_text = 'Count', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    width=800, height=400,
    annotations = list(fig_age2.layout.annotations) + 
    [go.layout.Annotation(
            y=-0.54,
            x=0.5,
            font=dict(
                size=14
            ),
            showarrow=False,
            text="Race",
            xref="paper",
            yref="paper"
        )
    ]
)

fig_age2.show()

## Where? Choropleth maps

The following maps show the geographical distributions of fatal encounters, considering information such as time, race and political majorities.

###Total Deaths

In [1141]:
kills_00_20 = police.groupby(['death_state']).size().div((df_pop_G['pop_2000'] + df_pop_G['pop_2010'] + df_pop_G['pop_2020'])/3, axis=0)*100000

kills_00_10 = round(police[police['death_date'].dt.year < 2011].groupby(['death_state']).size().div(df_pop_G['pop_2000'] + df_pop_G['pop_2010']/2, axis=0)*100000, 1)
kills_10_20 = round(police[(police['death_date'].dt.year > 2010) & (police['death_date'].dt.year < 2019)].groupby(['death_state']).size().div(df_pop_G['pop_2020'], axis=0)*100000, 1)

In [1142]:
fig1 = go.Figure(data=go.Choropleth(
    locations = police.groupby(['death_state']).size().index,
    z = police.groupby(['death_state']).size(), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    marker_line_color='white',
    colorbar_title = "Rate",
    colorbar_x=0.45
))

fig2 = go.Figure(data=go.Choropleth(
    locations=kills_00_20.index,
    z = kills_00_20, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    marker_line_color='white',
    colorbar_title = "Rate",
    colorbar_x=0.95
))

In [1143]:
from plotly.subplots import make_subplots
rows = 1
cols = 2
fig = make_subplots(
    rows=rows, cols=cols,
    specs = [[{'type': 'choropleth'} for c in np.arange(cols)] for r in np.arange(rows)],
    subplot_titles = ('Total Deaths','Deaths per 100000 inhabitants'),
    horizontal_spacing = 0.02)

fig.add_trace(fig1.data[0], row=1, col=1)

fig.add_trace(fig2.data[0], row=1, col=2)

fig.update_layout(
    height=600, width=1500,
    title_text = 'Number of deaths involving police', # tasso di morti per 100 000 abitanti
    **{'geo' + str(i) + '_scope': 'usa' for i in [''] + np.arange(2, rows*cols+1).tolist()},
    )

for index, trace in enumerate(fig.data):
    fig.data[index].hovertemplate = 'State: %{location}<br>Deaths: %{z:.1f}<extra></extra>'
fig.show()

###Total Deaths, by Decade

In [1144]:
fig10 = go.Figure(data=go.Choropleth(
    locations = kills_00_10.index,
    z = kills_00_10, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    zmin = 0,
    zmax = max(kills_10_20),
    marker_line_color='white',
    colorbar_title = "Rate"
))

fig20 = go.Figure(data=go.Choropleth(
    locations = kills_10_20.index,
    z = kills_10_20, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    zmin = 0,
    zmax = max(kills_10_20),
    marker_line_color='white',
    colorbar_title = "Rate",
))

In [1145]:
rows = 1
cols = 2
fig = make_subplots(
    rows=rows, cols=cols,
    specs = [[{'type': 'choropleth'} for c in np.arange(cols)] for r in np.arange(rows)],
    subplot_titles = ('from 2000 to 2010','from 2011 to 2020'),
    horizontal_spacing = 0.005)

fig.add_trace(fig10.data[0], row=1, col=1)

fig.add_trace(fig20.data[0], row=1, col=2)

fig.update_layout(
    height=600, width=1300,
    title_text = 'Fatal Encounter rate per 100000 inhabitants, by decade', # tasso di morti per 100 000 abitanti
    **{'geo' + str(i) + '_scope': 'usa' for i in [''] + np.arange(2, rows*cols+1).tolist()},
    )

for index, trace in enumerate(fig.data):
    fig.data[index].hovertemplate = 'State: %{location}<br>Deaths: %{z:.1f}<extra></extra>'
fig.show()

### Total Death, by Race

In [1146]:
kills_00_20_b = police[police['race']=='African-American/Black'].groupby(['death_state']).size().div(((df_pop_G['pop_2000'] + df_pop_G['pop_2010'] + df_pop_G['pop_2020'])/3)*df_pop_G['African-American/Black'], axis=0)*100000
kills_00_20_w = police[police['race']=='European-American/White'].groupby(['death_state']).size().div(((df_pop_G['pop_2000'] + df_pop_G['pop_2010'] + df_pop_G['pop_2020'])/3)*df_pop_G['European-American/White'], axis=0)*100000

kills_00_20_h = police[police['race']=='Hispanic/Latino'].groupby(['death_state']).size().div(((df_pop_G['pop_2000'] + df_pop_G['pop_2010'] + df_pop_G['pop_2020'])/3)*df_pop_G['Hispanic/Latino'], axis=0)*100000

In [1147]:
fig10 = go.Figure(data=go.Choropleth(
    locations = kills_00_20_b.index,
    z = kills_00_20_b, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    zmin = 0,
    zmax = max(kills_00_20_b),
    marker_line_color='white',
    colorbar_title = "Rate"
))

fig20 = go.Figure(data=go.Choropleth(
    locations = kills_00_20_w.index,
    z = kills_00_20_w, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    zmin = 0,
    zmax = max(kills_00_20_b),
    marker_line_color='white',
    colorbar_title = "Rate"
))


fig30 = go.Figure(data=go.Choropleth(
    locations = kills_00_20_h.index,
    z = kills_00_20_h, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    zmin = 0,
    zmax = max(kills_00_20_b),
    marker_line_color='white',
    colorbar_title = "Rate"
))

In [1148]:
rows = 1
cols = 3
fig = make_subplots(
    rows=rows, cols=cols,
    specs = [[{'type': 'choropleth'} for c in np.arange(cols)] for r in np.arange(rows)],
    subplot_titles = ('black','white', 'hispanic'),
    horizontal_spacing = 0.005)

fig.add_trace(fig10.data[0], row=1, col=1)

fig.add_trace(fig20.data[0], row=1, col=2)


fig.add_trace(fig30.data[0], row=1, col=3)

fig.update_layout(
    height=600, width=1500,
    title_text = 'Fatal Encounter rate per 100 000 inhabitants, normalized by race', # tasso di morti per 100 000 abitanti
    **{'geo' + str(i) + '_scope': 'usa' for i in [''] + np.arange(2, rows*cols+1).tolist()},
    )

for index, trace in enumerate(fig.data):
    fig.data[index].hovertemplate = 'State: %{location}<br>Deaths: %{z:.1f}<extra></extra>'
fig.show()

### Deaths by President

In [1149]:
police['president'] = police.death_date.dt.year.map(state_winner.set_index('year')['president'].to_dict())

In [1150]:
def grouper_by(df, president, race):
  df = police[(police['president']==president) & (police['race']==race)]
  if president == 'GEORGE W. BUSH':
    df = df.groupby(['death_state']).size().div(((df_pop_G['pop_2000'] + df_pop_G['pop_2010'])/2)*df_pop_G[race], axis=0)*100000/8
  elif president == 'BARACK OBAMA':
    df = df.groupby(['death_state']).size().div(((df_pop_G['pop_2010'] + df_pop_G['pop_2020'])/2)*df_pop_G[race], axis=0)*100000/8
  else: 
    df = df.groupby(['death_state']).size().div(df_pop_G['pop_2020']*df_pop_G[race], axis=0)*100000/4
  return df

kills_bush_b = grouper_by(police, 'GEORGE W. BUSH', 'African-American/Black')
kills_obama_b = grouper_by(police, 'BARACK OBAMA', 'African-American/Black')
kills_trump_b = grouper_by(police, 'DONALD TRUMP', 'African-American/Black')


kills_bush_w = grouper_by(police, 'GEORGE W. BUSH', 'European-American/White')
kills_obama_w = grouper_by(police, 'BARACK OBAMA', 'European-American/White')
kills_trump_w = grouper_by(police, 'DONALD TRUMP', 'European-American/White')

In [1151]:
palette_b = 'Blues' 
palette_w = 'Reds' 

def go_fig(df, z_max, colorbar):
  if colorbar == 'Black':
    palette = palette_b
    colorbar_pos = .98
  else:
    palette = palette_w
    colorbar_pos = -.05

  return go.Figure(data=go.Choropleth(
    locations = df.index,
    z = df, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = palette,
    zmin = 0,
    zmax = max(z_max),
    marker_line_color='#F5F5F5',
    colorbar_title = colorbar,
    colorbar_x = colorbar_pos)
  )

fig_bush_b = go_fig(kills_bush_b, kills_trump_b, 'Black')
fig_obama_b = go_fig(kills_obama_b, kills_trump_b, 'Black')
fig_trump_b = go_fig(kills_trump_b, kills_trump_b, 'Black')

fig_bush_w = go_fig(kills_bush_w, kills_trump_b, 'White')
fig_obama_w = go_fig(kills_obama_w, kills_trump_b, 'White')
fig_trump_w = go_fig(kills_trump_w, kills_trump_b, 'White')

In [1152]:
rows = 2
cols = 3
fig = make_subplots(
    rows=rows, cols=cols,
    specs = [[{'type': 'choropleth'} for c in np.arange(cols)] for r in np.arange(rows)],
    subplot_titles = ('Bush','Obama','Trump'),
    horizontal_spacing = 0.005)

fig.add_trace(fig_bush_b.data[0], row=1, col=1)
fig.add_trace(fig_obama_b.data[0], row=1, col=2)
fig.add_trace(fig_trump_b.data[0], row=1, col=3)

fig.add_trace(fig_bush_w.data[0], row=2, col=1)
fig.add_trace(fig_obama_w.data[0], row=2, col=2)
fig.add_trace(fig_trump_w.data[0], row=2, col=3)

fig.update_layout(
    height=600, width=1500,
    title_text = 'Death rate per 100 000 inhabitants, by President, by Race', # tasso di morti per 100 000 abitanti
    **{'geo' + str(i) + '_scope': 'usa' for i in [''] + np.arange(2, rows*cols+1).tolist()},
    )

for index, trace in enumerate(fig.data):
    fig.data[index].hovertemplate = 'State: %{location}<br>Deaths: %{z:.1f}<extra></extra>'

fig.show()

## Correlation Matrix

In [1153]:
p = police
p = p.merge(state_winner, how='inner', left_on=['death_state', p.death_date.dt.year], right_on=['state_po', 'year'])
p.drop(['statewinner',
        'president_x',
        'president_y', 
        'pres_party', 
        'death_state', 
        'death_date', 
        'death_city',
        'death_county', 
        'gender',
        'latitude', 
        'longitude',
        'year', 
        ], axis=1, inplace=True)

race_dict = dict(zip(police.race.unique(), range(len(police.race.unique()))))
p['race'] = p['race'].map(lambda s: race_dict.get(s))

mental_dict = dict(zip(police.mental_illness.unique(), range(len(police.mental_illness.unique()))))
p['mental_illness'] = p['mental_illness'].map(lambda s: mental_dict.get(s))

death_cause_dict = dict(zip(police.death_cause.unique(), range(len(police.death_cause.unique()))))
p['death_cause'] = p['death_cause'].map(lambda s: death_cause_dict.get(s))

use_of_force_dict = dict(zip(police.use_of_force.unique(), range(len(police.use_of_force.unique()))))
p['use_of_force'] = p['use_of_force'].map(lambda s: use_of_force_dict.get(s))

statewinner_dict = dict(zip(p.statewinner_party.unique(), range(len(p.statewinner_party.unique()))))
p['statewinner_party'] = p['statewinner_party'].map(lambda s: statewinner_dict.get(s))

state_dict = dict(zip(p.state_po.unique(), range(len(p.state_po.unique()))))
p['state_po'] = p['state_po'].map(lambda s: state_dict.get(s))

p.head()

,age,race,mental_illness,death_cause,use_of_force,trimester,death_description,dispositions_exclusions,state_po,statewinner_party
0,17,0,0,0,0,1,Samuel Knapp was allegedly driving a stolen ve...,Unreported,0,0
1,24,1,0,0,0,1,LaTanya Janelle McCoy's car was struck from be...,NaN,0,0
2,31,0,0,1,1,1,SWAT officers shot and killed Adrian Dolby sho...,Unreported,0,0
3,28,0,0,2,1,1,Joseph Gumpert stopped breathing after a scuff...,Unreported,0,0
4,24,2,0,1,1,1,Deputies shot and killed Sonny Daniel Diaz whe...,Unreported,0,0


In [1154]:
p['age'].astype('int')
p.corr(method='pearson').style.background_gradient(cmap='coolwarm')

,race,mental_illness,death_cause,use_of_force,trimester,state_po,statewinner_party
race,1.000000,-0.069854,0.013305,-0.036268,-0.010848,-0.157366,0.152728
mental_illness,-0.069854,1.000000,0.161445,0.151979,-0.007644,0.007320,0.023773
death_cause,0.013305,0.161445,1.000000,0.345137,0.007155,0.002518,0.006703
use_of_force,-0.036268,0.151979,0.345137,1.000000,-0.008788,0.020568,-0.021078
trimester,-0.010848,-0.007644,0.007155,-0.008788,1.000000,0.004780,0.010405
state_po,-0.157366,0.007320,0.002518,0.020568,0.004780,1.000000,-0.194709
statewinner_party,0.152728,0.023773,0.006703,-0.021078,0.010405,-0.194709,1.000000


#TASK IDEAS

*   Classification: Given a record for a fatal encounter, predict the Race of the deceased.
*   Regression: Given a location (city/county) predict the number of fatal encounters. 

Depending of the task that we will choose and its initial results, we could augment the dataset again, potentially with more information at the city/county level, especially economic ones (poverty, employment rate, etc...)

#Prova Extra

In [1155]:
police_desc = police.copy()

In [1156]:
eco = pd.read_csv(directory+'economy.csv')
del eco['Unnamed: 0']
eco['county'] = eco['county'].str.strip()
eco['state'] = eco['state'].str.strip()
eco[eco['county']=='District of Colu'] = eco[eco['county']=='District of Colu'].replace('District of Colu', 'District of Columbia')
eco[eco['state_po']=='ia'] = eco[eco['state_po']=='ia'].replace('ia', 'DC')
eco

,county,state,state_po,inequality2019,poverty2019
0,Autauga,Alabama,AL,14.94512,15.2
1,Baldwin,Alabama,AL,13.35974,10.4
2,Barbour,Alabama,AL,18.78242,30.7
3,Bibb,Alabama,AL,13.57075,18.1
4,Blount,Alabama,AL,14.46463,13.6
...,...,...,...,...,...
3132,Sweetwater,Wyoming,WY,12.22529,11.5
3133,Teton,Wyoming,WY,15.51484,6.1
3134,Uinta,Wyoming,WY,9.94881,11.3
3135,Washakie,Wyoming,WY,8.73628,10.5


In [1157]:
#Preprocessing
#Capitalizzare rimove 5000 righe con contea mismatchata!
police_desc['death_county'] = police_desc['death_county'].str.capitalize()
eco['county'] = eco['county'].str.capitalize()

police_desc['death_county'].replace({"Saint ": "St. ", 
                                     "St ":"St. ", 
                                     " county": ""}, regex=True, inplace=True)
police_desc['death_county'].replace({"mary":"mary's"}, regex=True, inplace=True)
police_desc['death_county'].replace({"mary's's":"mary's"}, regex=True, inplace=True)

In [1158]:
county1 = police_desc['death_county'].unique()
county2 = eco['county'].unique()
z = set(county1).intersection(set(county2))
print(len(z))
#Ci sono X "Contee" nel nostro dataset che non sono codificate in eco. Vediamo quali
diff = police_desc[~police_desc['death_county'].isin(county2)]
diff

1415


,age,gender,race,mental_illness,death_cause,use_of_force,death_date,trimester,death_city,death_county,death_state,latitude,longitude,death_description,dispositions_exclusions,president
136,27,Male,Hispanic/Latino,No,Vehicle,Vehicle/Pursuit,2000-02-25,1,Indianapolis,Indianapolis,IN,39.792601,-86.273002,Mario Cazarin was fleeing a traffic stop with ...,Unreported,GEORGE W. BUSH
257,31,Male,European-American/White,No,Gunshot,Yes,2000-04-18,2,Anchorage,Anchorage,AK,61.195789,-149.885740,Employees reported that a man was making faces...,Unreported,GEORGE W. BUSH
302,54,Male,African-American/Black,No,Gunshot,Yes,2000-05-06,2,Jackson,East feliciana,LA,30.837404,-91.217609,Reserve officer George Carter shot and killed ...,Guilty,GEORGE W. BUSH
365,20,Male,African-American/Black,No,Gunshot,Yes,2000-06-05,2,Jackson,East feliciana,LA,30.837404,-91.217609,Officers shot and killed Cleon Allen when he a...,NaN,GEORGE W. BUSH
369,68,Male,African-American/Black,No,Gunshot,Yes,2000-06-08,2,Baton rouge,East baton rouge,LA,30.423967,-91.139497,Officer Kyle Callihan shot and killed Henry Ta...,Justified,GEORGE W. BUSH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28121,33,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2020-05-20,2,Hammond,Tangipahoa,LA,30.519044,-90.524404,"At 2:15 a.m., troopers were called to assist t...",Pending,JOSEPH BIDEN
28192,33,Male,Other,No,Vehicle,Vehicle/Pursuit,2020-05-30,2,Oakdale,Evangeline,LA,30.795040,-92.572361,A trooper attempted a traffic stop on a 2003 S...,Pending,JOSEPH BIDEN
28235,37,Male,European-American/White,Yes,Gunshot,Yes,2020-06-06,2,Clarkrange,Fentress,TN,36.191998,-85.012951,"Around 7 p.m., police received a report of a m...",Pending,JOSEPH BIDEN
28289,35,Male,Hispanic/Latino,No,Gunshot,Yes,2020-06-20,2,Lake charles,Calcasieu,LA,30.118529,-93.236625,Deputies went to Michael Torres' home to serve...,Pending,JOSEPH BIDEN


In [1159]:
#Valori di county presenti nel nostro dataset che sono mismatchati. 
diff['death_county'].unique()

array(['Indianapolis', 'Anchorage', 'East feliciana', 'East baton rouge',
       'Bossier', 'St. landry', 'Northwest arctic',
       'Fairbanks north star', 'Rockbrdge', 'Ketchikan gateway',
       'Matanuska-susitna', 'Tangipahoa', 'West baton rouge',
       'St. john the baptist', 'Terrebonne', 'St. bernard',
       'Yukon koyukuk', 'Calcasieu', 'Kenai peninsula', 'Iberia',
       'Rapides', 'Evangeline', 'Nome', 'Acadia', 'North slope',
       'Morehouse', 'St. martin', 'Avoyelles', 'De soto', 'St. tammany',
       'Ascension', 'Concordia', 'Southeast fairbanks', 'Assumption',
       'Mcclean', 'Plaquemines', 'Dolton', 'Lafourche', 'Bedford city',
       'Athens-clarke', 'Natchitoches', 'Beauregard', 'Winn', 'Iberville',
       'Prince of wales hyder', 'Bethel', 'Sitka', 'Tensas', 'Petersburg',
       'Washentaw', 'Brooklyn', 'St. james', 'City of newport news',
       'Shively', 'Clarl', 'Denali', 'Beford', 'De kalb', 'La porte',
       'Dc', 'Du page', 'Park hill', 'Braziria', 'Os

In [1160]:
police_desc.loc[~police_desc['death_county'].isin(county2), 'death_county'] = np.vectorize(get_location)(police_desc[~police_desc['death_county'].isin(county2)]['latitude'], police_desc[~police_desc['death_county'].isin(county2)]['longitude'], 'county')

In [1161]:
police_desc['death_county'] = police_desc['death_county'].str.title()
eco['county'] = eco['county'].str.title()
county4 = eco['county'].unique()
police_desc['death_county'].replace({" County":""}, regex=True, inplace=True)
newcountyset = police_desc['death_county'].unique()
print(len(newcountyset))
print(len(county4))

w = set(newcountyset).intersection(set(county4))
police_desc[~police_desc['death_county'].isin(county4)]

1466
1873


,age,gender,race,mental_illness,death_cause,use_of_force,death_date,trimester,death_city,death_county,death_state,latitude,longitude,death_description,dispositions_exclusions,president
257,31,Male,European-American/White,No,Gunshot,Yes,2000-04-18,2,Anchorage,Anchorage,AK,61.195789,-149.885740,Employees reported that a man was making faces...,Unreported,GEORGE W. BUSH
665,19,Male,Hispanic/Latino,No,Gunshot,Yes,2000-10-01,4,Ambler,Northwest Arctic,AK,66.974627,-160.425502,An Alaska State Trooper shot and killed a robb...,Unreported,GEORGE W. BUSH
738,22,Male,Other,No,Gunshot,Yes,2000-10-29,4,Fairbanks,Fairbanks North Star,AK,64.836943,-147.739153,"According to a police statement, officers init...",Justified,GEORGE W. BUSH
850,38,Male,European-American/White,No,Gunshot,Yes,2000-12-31,4,Ketchikan,Ketchikan Gateway,AK,55.348303,-131.648924,A jail guard shot and killed an inmate after h...,Unreported,GEORGE W. BUSH
870,52,Male,European-American/White,Yes,Gunshot,Suicide,2001-01-09,1,Palmer,Matanuska-Susitna,AK,61.614685,-149.114386,Troopers were called to the home of Daniel E. ...,Suicide,GEORGE W. BUSH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27503,32,Male,European-American/White,No,Gunshot,Yes,2020-01-29,1,Kasilof,Kenai Peninsula,AK,60.300618,-151.283036,Troopers responded to a report of a gun being ...,Pending,JOSEPH BIDEN
27594,16,Male,Other,Drug or alcohol use,Gunshot,Yes,2020-02-16,1,Anchorage,Anchorage,AK,61.217452,-149.836798,Officers with the impaired driving enforcement...,Justified,JOSEPH BIDEN
27720,37,Male,European-American/White,No,Gunshot,Yes,2020-03-08,1,Wasilla,Matanuska-Susitna,AK,61.550603,-149.597009,"Around 7:15 p.m., troopers received a 911 call...",Pending,JOSEPH BIDEN
27731,32,Female,European-American/White,Yes,Gunshot,Yes,2020-03-11,1,Anchor point,Kenai Peninsula,AK,59.790826,-151.743148,Troopers responded to a 911 disconnect call at...,Pending,JOSEPH BIDEN


In [1162]:
police_desc[~police_desc['death_county'].isin(county4)]['death_county'].unique()

array(['Anchorage', 'Northwest Arctic', 'Fairbanks North Star',
       'Ketchikan Gateway', 'Matanuska-Susitna', 'Saint Helena Parish',
       'Saint Bernard Parish', 'Unorganized Borough', 'Kenai Peninsula',
       'North Slope', 'Athens-Clarke', 'Nome', 'Sitka',
       'Petersburg Borough', '', 'Newport News', 'Denali',
       'Oglala Lakota', 'Kusilvak Census Area'], dtype=object)

In [1163]:
police_desc[~police_desc['death_county'].isin(county4)]['death_county'].value_counts()

Anchorage               38
Matanuska-Susitna       22
Fairbanks North Star    18
Kenai Peninsula         14
Unorganized Borough      8
Athens-Clarke            6
                         6
Saint Bernard Parish     4
Northwest Arctic         3
Kusilvak Census Area     3
Oglala Lakota            3
Saint Helena Parish      2
North Slope              2
Nome                     2
Petersburg Borough       1
Sitka                    1
Newport News             1
Ketchikan Gateway        1
Denali                   1
Name: death_county, dtype: int64

In [1164]:
police_desc['death_county'].replace({"Anchorage": "Anchorage Borough/Municipality", 
                                     "Northwest Arctic":"Northwest Arctic Borough", 
                                     "Fairbanks North Star":"Fairbanks North Star Borough", 
                                     "Ketchikan Gateway":"Ketchikan Gateway Borough", 
                                     "Matanuska-Susitna":"Matanuska-Susitna Borough", 
                                     "Saint ": "St. ", 
                                     "Kenai Peninsula": "Kenai Peninsula Borough", 
                                     "North Slope": "North Slope Borough", 
                                     "Athens-Clarke": "Athens", 
                                     "Nome": "Nome Census Area", 
                                     "Sitka": "Sitka City And Borough",
                                     "Petersburg Borough":"Petersburg Census Area", 
                                     "Newport News":"Newport News City", 
                                     "Denali":"Denali Borough",
                                     "Oglala Lakota":"Dakota", 
                                     "Kusilvak Census Area":"Unorganized Borough",
                                     }, regex=True, inplace=True)

In [1165]:
police_desc[~police_desc['death_county'].isin(county4)]['death_county'].value_counts()

Newport News City City    15
Unorganized Borough       11
                           6
Name: death_county, dtype: int64

In [1166]:
police_desc[~police_desc['death_county'].isin(county4)]

,age,gender,race,mental_illness,death_cause,use_of_force,death_date,trimester,death_city,death_county,death_state,latitude,longitude,death_description,dispositions_exclusions,president
1746,22,Female,African-American/Black,No,Gunshot,Yes,2001-12-21,4,Newport news,Newport News City City,VA,37.152862,-76.547682,A Newport News police officer shot and killed ...,Unreported,GEORGE W. BUSH
1767,33,Male,African-American/Black,No,Gunshot,Yes,2001-12-27,4,Newport news,Newport News City City,VA,36.978238,-76.430119,Newport News police officers shot and killed T...,Unreported,GEORGE W. BUSH
2221,47,Male,European-American/White,Drug or alcohol use,Gunshot,Yes,2002-06-10,2,Kaltag,Unorganized Borough,AK,64.331897,-158.778550,"A trooper was called by Madros' wife, accusing...",Unreported,GEORGE W. BUSH
3398,19,Female,Other,No,Gunshot,Yes,2003-08-11,3,Nome,Unorganized Borough,AK,64.497389,-165.389989,Ivanoff disappeared from downtown Nome and was...,Guilty,GEORGE W. BUSH
4057,27,Male,African-American/Black,No,Vehicle,Vehicle/Pursuit,2004-03-25,1,Newport news,Newport News City City,VA,37.006425,-76.424533,Local police noticed an SUV at 2:45 a.m. speed...,Unreported,GEORGE W. BUSH
4058,25,Male,African-American/Black,No,Vehicle,Vehicle/Pursuit,2004-03-25,1,Newport news,Newport News City City,VA,37.006425,-76.424533,Local police noticed an SUV at 2:45 a.m. speed...,Unreported,GEORGE W. BUSH
4407,22,Male,Hispanic/Latino,No,Gunshot,Suicide,2004-07-25,3,Newport news,Newport News City City,VA,37.040146,-76.455864,Officer Christopher E. Miner saw Sanchez alleg...,Suicide,GEORGE W. BUSH
4520,54,Male,European-American/White,Yes,Gunshot,Yes,2004-09-07,3,Delta junction,Unorganized Borough,AK,64.152240,-145.842046,Troopers tried to serve arrest warrants on Dor...,Unreported,GEORGE W. BUSH
4634,22,Male,African-American/Black,No,Gunshot,Yes,2004-10-16,4,Newport news,Newport News City City,VA,36.978238,-76.430119,Santonio Dewayne Garrett was shot and killed b...,Unreported,GEORGE W. BUSH
7461,38,Male,African-American/Black,No,Gunshot,Yes,2007-02-18,1,Newport news,Newport News City City,VA,36.978627,-76.429425,Bail bondsmen came to arrest Harper on a bond ...,Justified,GEORGE W. BUSH


In [1167]:
police_desc['death_county'].replace({"Newport News City City": "Newport News City"}, regex=True, inplace=True)

In [1168]:
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_city']=='Washington'), 'death_county'] = 'District Of Columbia'

In [1169]:
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_county']=='') & (police_desc['death_city']=='New york'), 'death_county'] = 'New York'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_county']=='') & (police_desc['death_city']=='Brooklyn'), 'death_county'] = 'Kings'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_county']=='') & (police_desc['death_city']=='Chesapeake'), 'death_county'] = 'Kanawa'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_county']=='') & (police_desc['death_city']=='Virginia beach'), 'death_county'] = 'Virginia Beach City'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_county']=='') & (police_desc['death_city']=='Lynchburg'), 'death_county'] = 'Lynchburg City'

In [1170]:
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_city']!='Washington')]

,age,gender,race,mental_illness,death_cause,use_of_force,death_date,trimester,death_city,death_county,death_state,latitude,longitude,death_description,dispositions_exclusions,president
2221,47,Male,European-American/White,Drug or alcohol use,Gunshot,Yes,2002-06-10,2,Kaltag,Unorganized Borough,AK,64.331897,-158.778550,"A trooper was called by Madros' wife, accusing...",Unreported,GEORGE W. BUSH
3398,19,Female,Other,No,Gunshot,Yes,2003-08-11,3,Nome,Unorganized Borough,AK,64.497389,-165.389989,Ivanoff disappeared from downtown Nome and was...,Guilty,GEORGE W. BUSH
4520,54,Male,European-American/White,Yes,Gunshot,Yes,2004-09-07,3,Delta junction,Unorganized Borough,AK,64.152240,-145.842046,Troopers tried to serve arrest warrants on Dor...,Unreported,GEORGE W. BUSH
9454,27,Male,European-American/White,Yes,Gunshot,Yes,2008-09-24,3,Craig,Unorganized Borough,AK,55.476389,-133.148333,Troopers say Craig police contacted them Wedne...,Unreported,BARACK OBAMA
9662,34,Male,European-American/White,Yes,Gunshot,Yes,2008-12-05,4,Bethel,Unorganized Borough,AK,60.805078,-161.799580,Police told them officers were responding to a...,Unreported,BARACK OBAMA
14847,24,Male,African-American/Black,Drug or alcohol use,Gunshot,Yes,2012-10-02,4,Bethel,Unorganized Borough,AK,60.792020,-161.743239,Bethel Police Department responded to a distur...,Unreported,BARACK OBAMA
21428,24,Male,Other,Drug or alcohol use,Gunshot,Suicide,2016-09-07,3,Chevak,Unorganized Borough,AK,61.529445,-165.591880,Village Police Chief Derek McDonald responded ...,Suicide,DONALD TRUMP
22479,49,Female,Other,No,Vehicle,Vehicle/Pursuit,2017-04-26,2,Bethel,Unorganized Borough,AK,60.785279,-161.801893,Sargent Kadri Lamani collided with two pedestr...,Justified,DONALD TRUMP
24009,27,Male,Other,No,Gunshot,Yes,2018-03-08,1,Pilot station,Unorganized Borough,AK,61.942059,-162.881907,"Dwight Heckman allegedly assaulted a woman, st...",Pending,DONALD TRUMP
24642,38,Male,Other,No,Gunshot,Yes,2018-07-06,3,Copper center,Unorganized Borough,AK,61.982836,-145.353134,"Eric Hash, was killed at about 3:15 a.m. Troop...",Pending,DONALD TRUMP


In [1171]:
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_city']=='Kaltag'), 'death_county'] = 'Yukon-Koyukuk Census Area'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_city']=='Nome'), 'death_county'] = 'Nome Census Area'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_city']=='Delta junction'), 'death_county'] = 'Southeast Fairbanks Census Area'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_city']=='Craig'), 'death_county'] = 'Prince Of Wales-Hyder Census Area'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_city']=='Bethel'), 'death_county'] = 'Bethel Census Area'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_city']=='Chevak'), 'death_county'] = 'Bethel Census Area'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_city']=='Pilot station'), 'death_county'] = 'Bethel Census Area'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_city']=='Copper center'), 'death_county'] = 'Valdez-Cordova Census Area'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_city']=="Saint mary's"), 'death_county'] = 'Bethel Census Area'
police_desc.loc[(~police_desc['death_county'].isin(county4)) & (police_desc['death_city']=='Chesapeake'), 'death_county'] = 'Kanawha'


In [1172]:
police_desc['death_county'] = police_desc['death_county']+', '+police_desc['death_state']

In [1173]:
police_desc['death_city'] = police_desc['death_city']+', '+police_desc['death_county']
police_desc

,age,gender,race,mental_illness,death_cause,use_of_force,death_date,trimester,death_city,death_county,death_state,latitude,longitude,death_description,dispositions_exclusions,president
0,17,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Willits, Mendocino, CA","Mendocino, CA",CA,39.470883,-123.361751,Samuel Knapp was allegedly driving a stolen ve...,Unreported,GEORGE W. BUSH
1,21,Male,African-American/Black,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Detroit, Wayne, MI","Wayne, MI",MI,42.404526,-83.092274,Two Detroit men killed when their car crashed ...,Unreported,GEORGE W. BUSH
2,19,Male,African-American/Black,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Detroit, Wayne, MI","Wayne, MI",MI,42.404526,-83.092274,Two Detroit men killed when their car crashed ...,Unreported,GEORGE W. BUSH
3,23,Male,Hispanic/Latino,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Carlsbad, Eddy, NM","Eddy, NM",NM,32.450080,-104.237643,A motorcycle was allegedly being driven errati...,Unreported,GEORGE W. BUSH
4,53,Male,African-American/Black,No,Gunshot,Yes,2000-01-02,1,"Ellenwood, Dekalb, GA","Dekalb, GA",GA,33.645164,-84.229413,"Darren Mayfield, a DeKalb County sheriff's dep...",Guilty,GEORGE W. BUSH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28329,40,Female,European-American/White,No,Gunshot,Yes,2020-06-27,2,"Port richey, Pasco, FL","Pasco, FL",FL,28.286752,-82.705168,Bonnie Figueroa-Ortiz was reported missing. De...,Pending,JOSEPH BIDEN
28330,18,Male,European-American/White,No,Vehicle,No,2020-06-27,2,"Miami, Miami-Dade, FL","Miami-Dade, FL",FL,25.739723,-80.157708,A Miami-Dade police officer in a marked cruise...,Pending,JOSEPH BIDEN
28331,42,Female,European-American/White,No,Vehicle,Vehicle/Pursuit,2020-06-27,2,"Petersburg, Prince George, VA","Prince George, VA",VA,37.214684,-77.363477,"Around 10:30 a.m., a Prince George officer clo...",Pending,JOSEPH BIDEN
28332,25,Female,European-American/White,No,Vehicle,Vehicle/Pursuit,2020-06-27,2,"Petersburg, Prince George, VA","Prince George, VA",VA,37.214684,-77.363477,"Around 10:30 a.m., a Prince George officer clo...",Pending,JOSEPH BIDEN


In [1174]:
eco['county'] = eco['county']+', '+eco['state_po']
eco

,county,state,state_po,inequality2019,poverty2019
0,"Autauga, AL",Alabama,AL,14.94512,15.2
1,"Baldwin, AL",Alabama,AL,13.35974,10.4
2,"Barbour, AL",Alabama,AL,18.78242,30.7
3,"Bibb, AL",Alabama,AL,13.57075,18.1
4,"Blount, AL",Alabama,AL,14.46463,13.6
...,...,...,...,...,...
3132,"Sweetwater, WY",Wyoming,WY,12.22529,11.5
3133,"Teton, WY",Wyoming,WY,15.51484,6.1
3134,"Uinta, WY",Wyoming,WY,9.94881,11.3
3135,"Washakie, WY",Wyoming,WY,8.73628,10.5


#JOIN AND EXPORT CSV

In [1175]:
police_desc.head()

,age,gender,race,mental_illness,death_cause,use_of_force,death_date,trimester,death_city,death_county,death_state,latitude,longitude,death_description,dispositions_exclusions,president
0,17,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Willits, Mendocino, CA","Mendocino, CA",CA,39.470883,-123.361751,Samuel Knapp was allegedly driving a stolen ve...,Unreported,GEORGE W. BUSH
1,21,Male,African-American/Black,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Detroit, Wayne, MI","Wayne, MI",MI,42.404526,-83.092274,Two Detroit men killed when their car crashed ...,Unreported,GEORGE W. BUSH
2,19,Male,African-American/Black,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Detroit, Wayne, MI","Wayne, MI",MI,42.404526,-83.092274,Two Detroit men killed when their car crashed ...,Unreported,GEORGE W. BUSH
3,23,Male,Hispanic/Latino,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Carlsbad, Eddy, NM","Eddy, NM",NM,32.450080,-104.237643,A motorcycle was allegedly being driven errati...,Unreported,GEORGE W. BUSH
4,53,Male,African-American/Black,No,Gunshot,Yes,2000-01-02,1,"Ellenwood, Dekalb, GA","Dekalb, GA",GA,33.645164,-84.229413,"Darren Mayfield, a DeKalb County sheriff's dep...",Guilty,GEORGE W. BUSH


In [1176]:
eco.rename(columns={'county':'death_county'}, inplace=True)
eco.head()

,death_county,state,state_po,inequality2019,poverty2019
0,"Autauga, AL",Alabama,AL,14.94512,15.2
1,"Baldwin, AL",Alabama,AL,13.35974,10.4
2,"Barbour, AL",Alabama,AL,18.78242,30.7
3,"Bibb, AL",Alabama,AL,13.57075,18.1
4,"Blount, AL",Alabama,AL,14.46463,13.6


In [1177]:
df_final = police_desc.merge(eco, on='death_county')

In [1178]:
df_final.drop(columns='president', inplace=True)

In [1179]:
df_final.drop(columns=['state', 'death_state'], inplace=True)
df_final

,age,gender,race,mental_illness,death_cause,use_of_force,death_date,trimester,death_city,death_county,latitude,longitude,death_description,dispositions_exclusions,state_po,inequality2019,poverty2019
0,17,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Willits, Mendocino, CA","Mendocino, CA",39.470883,-123.361751,Samuel Knapp was allegedly driving a stolen ve...,Unreported,CA,14.70610,17.8
1,35,Male,European-American/White,No,Gunshot,Yes,2003-03-07,1,"Ukiah, Mendocino, CA","Mendocino, CA",39.133730,-123.197032,Police shot Neal Allen Beckman when he alleged...,Unreported,CA,14.70610,17.8
2,41,Male,Other,No,Gunshot,Yes,2005-10-23,4,"Hopland, Mendocino, CA","Mendocino, CA",38.982178,-123.058482,"Deputies shot and killed Godfrey Luke John, wh...",Justified,CA,14.70610,17.8
3,20,Male,Hispanic/Latino,No,Gunshot,Yes,2007-04-02,2,"Ukiah, Mendocino, CA","Mendocino, CA",39.124932,-123.208113,"Sgt. Greg Heitkamp, Sgt. Justin Wyatt, Detecti...",Justified,CA,14.70610,17.8
4,40,Male,Hispanic/Latino,No,Gunshot,Yes,2007-12-29,4,"Ukiah, Mendocino, CA","Mendocino, CA",39.152626,-123.213084,Sgt. Sean Kaeser and Officer Chris Gordon shot...,Justified,CA,14.70610,17.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27990,46,Male,European-American/White,No,Gunshot,Yes,2020-05-30,2,"Fountain, Calhoun, FL","Calhoun, FL",30.515100,-85.348315,"Derrick Thompson, 46, allegedly fatally shot K...",Pending,FL,14.20742,14.7
27991,23,Male,Hispanic/Latino,No,Gunshot,Yes,2020-05-30,2,"Fountain, Calhoun, FL","Calhoun, FL",30.515100,-85.348315,"Derrick Thompson, 46, allegedly fatally shot K...",Pending,FL,14.20742,14.7
27992,37,Male,European-American/White,Yes,Gunshot,Yes,2020-06-06,2,"Clarkrange, Fentress, TN","Fentress, TN",36.191998,-85.012951,"Around 7 p.m., police received a report of a m...",Pending,TN,10.83375,20.6
27993,28,Male,European-American/White,No,Gunshot,Yes,2020-06-11,2,"Dowelltown, Dekalb, TN","Dekalb, TN",35.996617,-85.878333,Gregorio Cruz Vanloo was a suspect in a “shots...,Pending,TN,16.12409,18.4


In [1180]:
df_pop.head()

,state_po,pop_2000,pop_2010,pop_2020,State,European-American/White,African-American/Black,Hispanic/Latino,Other,areaKM,density2020
0,AL,4447207,4779736,5024279,Alabama,0.6552,0.2649,0.0428,0.0371,135765,37.007174
1,AK,626933,710231,733391,Alaska,0.6063,0.0310,0.0705,0.2922,1717854,0.426923
2,AZ,5130247,6392017,7151502,Arizona,0.5471,0.0421,0.3134,0.0974,295234,24.223165
3,AR,2673293,2915918,3011524,Arkansas,0.7243,0.1523,0.0747,0.0487,137732,21.865100
4,CA,33871653,37253956,39538223,California,0.3718,0.0552,0.3902,0.1828,423970,93.257124


In [1181]:
df_final = df_final.merge(df_pop, on='state_po')
df_final

,age,gender,race,mental_illness,death_cause,use_of_force,death_date,trimester,death_city,death_county,latitude,longitude,death_description,dispositions_exclusions,state_po,inequality2019,poverty2019,pop_2000,pop_2010,pop_2020,State,European-American/White,African-American/Black,Hispanic/Latino,Other,areaKM,density2020
0,17,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Willits, Mendocino, CA","Mendocino, CA",39.470883,-123.361751,Samuel Knapp was allegedly driving a stolen ve...,Unreported,CA,14.70610,17.8,33871653,37253956,39538223,California,0.3718,0.0552,0.3902,0.1828,423970,93.257124
1,35,Male,European-American/White,No,Gunshot,Yes,2003-03-07,1,"Ukiah, Mendocino, CA","Mendocino, CA",39.133730,-123.197032,Police shot Neal Allen Beckman when he alleged...,Unreported,CA,14.70610,17.8,33871653,37253956,39538223,California,0.3718,0.0552,0.3902,0.1828,423970,93.257124
2,41,Male,Other,No,Gunshot,Yes,2005-10-23,4,"Hopland, Mendocino, CA","Mendocino, CA",38.982178,-123.058482,"Deputies shot and killed Godfrey Luke John, wh...",Justified,CA,14.70610,17.8,33871653,37253956,39538223,California,0.3718,0.0552,0.3902,0.1828,423970,93.257124
3,20,Male,Hispanic/Latino,No,Gunshot,Yes,2007-04-02,2,"Ukiah, Mendocino, CA","Mendocino, CA",39.124932,-123.208113,"Sgt. Greg Heitkamp, Sgt. Justin Wyatt, Detecti...",Justified,CA,14.70610,17.8,33871653,37253956,39538223,California,0.3718,0.0552,0.3902,0.1828,423970,93.257124
4,40,Male,Hispanic/Latino,No,Gunshot,Yes,2007-12-29,4,"Ukiah, Mendocino, CA","Mendocino, CA",39.152626,-123.213084,Sgt. Sean Kaeser and Officer Chris Gordon shot...,Justified,CA,14.70610,17.8,33871653,37253956,39538223,California,0.3718,0.0552,0.3902,0.1828,423970,93.257124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27990,10,Female,European-American/White,No,Vehicle,Vehicle/Pursuit,2018-07-04,3,"Yankton, Yankton, SD","Yankton, SD",42.879958,-97.313419,A Yankton police officer was trying to stop an...,Unreported,SD,12.89777,11.1,754858,814180,886667,South Dakota,0.8196,0.0197,0.0379,0.1228,199371,4.447322
27991,27,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2018-07-04,3,"Yankton, Yankton, SD","Yankton, SD",42.879958,-97.313419,A Yankton police officer was trying to stop an...,Unreported,SD,12.89777,11.1,754858,814180,886667,South Dakota,0.8196,0.0197,0.0379,0.1228,199371,4.447322
27992,29,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2018-07-04,3,"Yankton, Yankton, SD","Yankton, SD",42.879958,-97.313419,A Yankton police officer was trying to stop an...,Unreported,SD,12.89777,11.1,754858,814180,886667,South Dakota,0.8196,0.0197,0.0379,0.1228,199371,4.447322
27993,40,Female,European-American/White,No,Vehicle,Vehicle/Pursuit,2018-07-04,3,"Yankton, Yankton, SD","Yankton, SD",42.879958,-97.313419,A Yankton police officer was trying to stop an...,Unreported,SD,12.89777,11.1,754858,814180,886667,South Dakota,0.8196,0.0197,0.0379,0.1228,199371,4.447322


In [1182]:
df_final['pop'] = 0
df_final.loc[df_final['death_date'].dt.year==2000, 'pop'] = df_final['pop_2000']
df_final.loc[df_final['death_date'].dt.year==2010, 'pop'] = df_final['pop_2010'] 
df_final.loc[df_final['death_date'].dt.year==2020, 'pop'] = df_final['pop_2020'] 
df_final.loc[(df_final['death_date'].dt.year<2010) & (df_final['death_date'].dt.year>2000), 'pop'] = (df_final['pop_2000']+df_final['pop_2010'])//2
df_final.loc[(df_final['death_date'].dt.year>2010) & (df_final['death_date'].dt.year<2020), 'pop'] = (df_final['pop_2010']+df_final['pop_2020'])//2

df_final['density'] = (df_final['pop'] / df_final['areaKM'])

In [1183]:
df_final.drop(columns=['pop_2000', 'pop_2010','pop_2020','density2020', 'State'], inplace=True)
df_final

,age,gender,race,mental_illness,death_cause,use_of_force,death_date,trimester,death_city,death_county,latitude,longitude,death_description,dispositions_exclusions,state_po,inequality2019,poverty2019,European-American/White,African-American/Black,Hispanic/Latino,Other,areaKM,pop,density
0,17,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Willits, Mendocino, CA","Mendocino, CA",39.470883,-123.361751,Samuel Knapp was allegedly driving a stolen ve...,Unreported,CA,14.70610,17.8,0.3718,0.0552,0.3902,0.1828,423970,33871653,79.891627
1,35,Male,European-American/White,No,Gunshot,Yes,2003-03-07,1,"Ukiah, Mendocino, CA","Mendocino, CA",39.133730,-123.197032,Police shot Neal Allen Beckman when he alleged...,Unreported,CA,14.70610,17.8,0.3718,0.0552,0.3902,0.1828,423970,35562804,83.880473
2,41,Male,Other,No,Gunshot,Yes,2005-10-23,4,"Hopland, Mendocino, CA","Mendocino, CA",38.982178,-123.058482,"Deputies shot and killed Godfrey Luke John, wh...",Justified,CA,14.70610,17.8,0.3718,0.0552,0.3902,0.1828,423970,35562804,83.880473
3,20,Male,Hispanic/Latino,No,Gunshot,Yes,2007-04-02,2,"Ukiah, Mendocino, CA","Mendocino, CA",39.124932,-123.208113,"Sgt. Greg Heitkamp, Sgt. Justin Wyatt, Detecti...",Justified,CA,14.70610,17.8,0.3718,0.0552,0.3902,0.1828,423970,35562804,83.880473
4,40,Male,Hispanic/Latino,No,Gunshot,Yes,2007-12-29,4,"Ukiah, Mendocino, CA","Mendocino, CA",39.152626,-123.213084,Sgt. Sean Kaeser and Officer Chris Gordon shot...,Justified,CA,14.70610,17.8,0.3718,0.0552,0.3902,0.1828,423970,35562804,83.880473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27990,10,Female,European-American/White,No,Vehicle,Vehicle/Pursuit,2018-07-04,3,"Yankton, Yankton, SD","Yankton, SD",42.879958,-97.313419,A Yankton police officer was trying to stop an...,Unreported,SD,12.89777,11.1,0.8196,0.0197,0.0379,0.1228,199371,850423,4.265530
27991,27,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2018-07-04,3,"Yankton, Yankton, SD","Yankton, SD",42.879958,-97.313419,A Yankton police officer was trying to stop an...,Unreported,SD,12.89777,11.1,0.8196,0.0197,0.0379,0.1228,199371,850423,4.265530
27992,29,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2018-07-04,3,"Yankton, Yankton, SD","Yankton, SD",42.879958,-97.313419,A Yankton police officer was trying to stop an...,Unreported,SD,12.89777,11.1,0.8196,0.0197,0.0379,0.1228,199371,850423,4.265530
27993,40,Female,European-American/White,No,Vehicle,Vehicle/Pursuit,2018-07-04,3,"Yankton, Yankton, SD","Yankton, SD",42.879958,-97.313419,A Yankton police officer was trying to stop an...,Unreported,SD,12.89777,11.1,0.8196,0.0197,0.0379,0.1228,199371,850423,4.265530


In [1184]:
state_winner.head()

,year,state_po,statewinner_party,statewinner,president,pres_party
0,2000,AK,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
28,2000,ND,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
29,2000,NE,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
30,2000,NH,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
31,2000,NJ,DEMOCRAT,AL GORE,GEORGE W. BUSH,REPUBLICAN


In [1185]:
df_final['year'] = df_final['death_date'].dt.year
df_final = df_final.merge(state_winner, on=['year', 'state_po'])
df_final

,age,gender,race,mental_illness,death_cause,use_of_force,death_date,trimester,death_city,death_county,latitude,longitude,death_description,dispositions_exclusions,state_po,inequality2019,poverty2019,European-American/White,African-American/Black,Hispanic/Latino,Other,areaKM,pop,density,year,statewinner_party,statewinner,president,pres_party
0,17,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Willits, Mendocino, CA","Mendocino, CA",39.470883,-123.361751,Samuel Knapp was allegedly driving a stolen ve...,Unreported,CA,14.70610,17.8,0.3718,0.0552,0.3902,0.1828,423970,33871653,79.891627,2000,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
1,24,Female,African-American/Black,No,Vehicle,Vehicle/Pursuit,2000-01-02,1,"Sacramento, Sacramento, CA","Sacramento, CA",38.473949,-121.433776,LaTanya Janelle McCoy's car was struck from be...,NaN,CA,14.53699,14.7,0.3718,0.0552,0.3902,0.1828,423970,33871653,79.891627,2000,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
2,40,Male,European-American/White,Yes,Gunshot,Suicide,2000-02-05,1,"Mather, Sacramento, CA","Sacramento, CA",38.572804,-121.295151,Andrea John Ketcherside reportedly shot himsel...,Suicide,CA,14.53699,14.7,0.3718,0.0552,0.3902,0.1828,423970,33871653,79.891627,2000,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
3,43,Male,European-American/White,No,Gunshot,Yes,2000-04-05,2,"Sacramento, Sacramento, CA","Sacramento, CA",38.567156,-121.495906,Marvin Edward Curry was found dead inside his ...,NaN,CA,14.53699,14.7,0.3718,0.0552,0.3902,0.1828,423970,33871653,79.891627,2000,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
4,47,Male,European-American/White,No,Gunshot,Yes,2000-07-17,3,"Carmichael, Sacramento, CA","Sacramento, CA",38.595126,-121.346210,Deputies shot and killed Taylor Bradley when h...,NaN,CA,14.53699,14.7,0.3718,0.0552,0.3902,0.1828,423970,33871653,79.891627,2000,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27994,16,Male,European-American/White,No,Gunshot,Yes,2007-01-26,1,"Huron, Beadle, SD","Beadle, SD",44.349448,-98.224463,According to the Argus Leader: Officers respon...,Justified,SD,10.27151,16.2,0.8196,0.0197,0.0379,0.1228,199371,784519,3.934970,2007,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
27995,42,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2007-06-14,2,"Bridgewater, Mccook, SD","Mccook, SD",43.487689,-97.312411,"Lyle Doering, the father of Deanna Mogck and T...",Unreported,SD,9.38396,9.7,0.8196,0.0197,0.0379,0.1228,199371,784519,3.934970,2007,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
27996,9,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2007-06-14,2,"Bridgewater, Mccook, SD","Mccook, SD",43.487689,-97.312411,The father of Deanna Mogck and Trevor Doering ...,Unreported,SD,9.38396,9.7,0.8196,0.0197,0.0379,0.1228,199371,784519,3.934970,2007,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN
27997,13,Female,European-American/White,No,Vehicle,Vehicle/Pursuit,2007-06-14,2,"Bridgewater, Mccook, SD","Mccook, SD",43.487689,-97.312411,The father of Deanna Mogck and Trevor Doering ...,Unreported,SD,9.38396,9.7,0.8196,0.0197,0.0379,0.1228,199371,784519,3.934970,2007,REPUBLICAN,GEORGE W. BUSH,GEORGE W. BUSH,REPUBLICAN


In [1186]:
df_final.drop(columns=['year'], inplace=True)


In [1187]:
df_final['statewinner_party'] = df_final['statewinner_party'].str.title()
df_final['statewinner'] = df_final['statewinner'].str.title()
df_final['president'] = df_final['president'].str.title()
df_final['pres_party'] = df_final['pres_party'].str.title()

df_final

,age,gender,race,mental_illness,death_cause,use_of_force,death_date,trimester,death_city,death_county,latitude,longitude,death_description,dispositions_exclusions,state_po,inequality2019,poverty2019,European-American/White,African-American/Black,Hispanic/Latino,Other,areaKM,pop,density,statewinner_party,statewinner,president,pres_party
0,17,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2000-01-01,1,"Willits, Mendocino, CA","Mendocino, CA",39.470883,-123.361751,Samuel Knapp was allegedly driving a stolen ve...,Unreported,CA,14.70610,17.8,0.3718,0.0552,0.3902,0.1828,423970,33871653,79.891627,Republican,George W. Bush,George W. Bush,Republican
1,24,Female,African-American/Black,No,Vehicle,Vehicle/Pursuit,2000-01-02,1,"Sacramento, Sacramento, CA","Sacramento, CA",38.473949,-121.433776,LaTanya Janelle McCoy's car was struck from be...,NaN,CA,14.53699,14.7,0.3718,0.0552,0.3902,0.1828,423970,33871653,79.891627,Republican,George W. Bush,George W. Bush,Republican
2,40,Male,European-American/White,Yes,Gunshot,Suicide,2000-02-05,1,"Mather, Sacramento, CA","Sacramento, CA",38.572804,-121.295151,Andrea John Ketcherside reportedly shot himsel...,Suicide,CA,14.53699,14.7,0.3718,0.0552,0.3902,0.1828,423970,33871653,79.891627,Republican,George W. Bush,George W. Bush,Republican
3,43,Male,European-American/White,No,Gunshot,Yes,2000-04-05,2,"Sacramento, Sacramento, CA","Sacramento, CA",38.567156,-121.495906,Marvin Edward Curry was found dead inside his ...,NaN,CA,14.53699,14.7,0.3718,0.0552,0.3902,0.1828,423970,33871653,79.891627,Republican,George W. Bush,George W. Bush,Republican
4,47,Male,European-American/White,No,Gunshot,Yes,2000-07-17,3,"Carmichael, Sacramento, CA","Sacramento, CA",38.595126,-121.346210,Deputies shot and killed Taylor Bradley when h...,NaN,CA,14.53699,14.7,0.3718,0.0552,0.3902,0.1828,423970,33871653,79.891627,Republican,George W. Bush,George W. Bush,Republican
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27994,16,Male,European-American/White,No,Gunshot,Yes,2007-01-26,1,"Huron, Beadle, SD","Beadle, SD",44.349448,-98.224463,According to the Argus Leader: Officers respon...,Justified,SD,10.27151,16.2,0.8196,0.0197,0.0379,0.1228,199371,784519,3.934970,Republican,George W. Bush,George W. Bush,Republican
27995,42,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2007-06-14,2,"Bridgewater, Mccook, SD","Mccook, SD",43.487689,-97.312411,"Lyle Doering, the father of Deanna Mogck and T...",Unreported,SD,9.38396,9.7,0.8196,0.0197,0.0379,0.1228,199371,784519,3.934970,Republican,George W. Bush,George W. Bush,Republican
27996,9,Male,European-American/White,No,Vehicle,Vehicle/Pursuit,2007-06-14,2,"Bridgewater, Mccook, SD","Mccook, SD",43.487689,-97.312411,The father of Deanna Mogck and Trevor Doering ...,Unreported,SD,9.38396,9.7,0.8196,0.0197,0.0379,0.1228,199371,784519,3.934970,Republican,George W. Bush,George W. Bush,Republican
27997,13,Female,European-American/White,No,Vehicle,Vehicle/Pursuit,2007-06-14,2,"Bridgewater, Mccook, SD","Mccook, SD",43.487689,-97.312411,The father of Deanna Mogck and Trevor Doering ...,Unreported,SD,9.38396,9.7,0.8196,0.0197,0.0379,0.1228,199371,784519,3.934970,Republican,George W. Bush,George W. Bush,Republican


In [1188]:
from google.colab import files
df_final.to_csv('df_final.csv')
files.download('df_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install jovian --upgrade --quiet
import jovian
jovian.commit(project="dataunderstandingv2")